# Maximum Number of Employees Reached

Write a query that returns every employee that has ever worked for the company. For each employee, calculate the greatest number of employees that worked for the company during their tenure and the first date that number was reached. The termination date of an employee should not be counted as a working day. 



Your output should have the employee ID, greatest number of employees that worked for the company during the employee's tenure, and first date that number was reached.

# Approaches


- Get those with hire_date & create column = 1, get those with termination_date & create column = -1 .. finally, concat those 2 dataframes

- CumSum columns

- Fetch earliest date where max employee was reached

- Order by id, maxemp, and date

In [134]:
# Import your libraries
import pandas as pd
import numpy as np
import datetime as dt
from getdata_fromhtml import getdata_fromhtml
# Start writing code
path = '/home/kassa/Downloads/StrataScratch_files/uber_employee.html'

df = getdata_fromhtml(path) #uber_employees#.head()
df = df.replace(r'^\s*$', 'NaT', regex=True)

#start date of an employee 
start = df['hire_date'].to_frame('date')
start['value'] = 1 # get hired 
#start

end_date = df['termination_date'].to_frame('date').dropna()
end_date['value'] = -1
#end_date
# Concatinate the dates 
dates = pd.concat([start, end_date], ignore_index = True)

#count the number of employess for each day 
dates = dates.groupby('date')['value'].sum().reset_index()

# Order the dataset in ascending order according to the date 
dates = dates.sort_values('date')
dates['cum_emp_count'] = dates['value'].cumsum()

# Find the 
def max_employe_min_date(x):
    if df.at[x, 'termination_date'] is pd.NaT:
        end_date = dt.datetime.today()
    else:
        end_date = df.at[x, 'termination_date']
    startdate = df.at[x, 'hire_date']
    max_no_emp = dates[dates['date'].between(startdate, end_date)]['cum_emp_count'].max()
    df.at[x, 'max_cont'] = max_no_emp
    # Find the earliest date that the max_no_emp is obtained 
    early_date = dates[(dates['cum_emp_count'] == max_no_emp) & (dates['date'].between(startdate, end_date))]['date'].min()
    df.at[x, 'earliest_date'] = early_date
    return df
    
for i in range(0, df.shape[0]):
    max_employe_min_date(i)
#lis = list(range(0, df.shape[0]))
#df_new = pd.DataFrame(map(max_employe_min_date, lis))
#print(df_new)
#df_new = df_new[['id', 'max_cont', 'earliest_date']]
result = df[['id', 'max_cont', 'earliest_date']] 
    
result.head()

,id,max_cont,earliest_date
0,1,57.0,2015-04-10
1,2,58.0,2017-02-03
2,3,58.0,2017-02-03
3,4,48.0,2013-12-23
4,5,58.0,2017-02-03


# Distance Per Dollar
- Hard

You’re given a dataset of uber rides with the traveling distance (‘distance_to_travel’) and cost (‘monetary_cost’) for each ride. For each date, find the difference between the distance-per-dollar for that date and the average distance-per-dollar for that year-month. Distance-per-dollar is defined as the distance traveled divided by the cost of the ride.



The output should include the year-month (YYYY-MM) and the absolute average difference in distance-per-dollar (Absolute value to be rounded to the 2nd decimal). 

You should also count both success and failed request_status as the distance and cost values are populated for all ride requests. Also, assume that all dates are unique in the dataset. Order your results by earliest request date first.

# Approaches

- Find the distance-per-dollar by dividing the distance and cost columns 

- Add a month column extracted from the request_date column to help with grouping the distance-per-dollar by month 

- Use a window function to find the average monthly distance-per-dollar for each month

- Calculate the absolute difference between the distance_to_cost for each request_date and month



In [135]:
# Import your libraries
import pandas as pd
import datetime as dt

path = '/home/kassa/Downloads/StrataScratch_files/uber_requests_logs.html'
# Start writing code
df = getdata_fromhtml(path)
cnts = df[df['request_status'] == 'success'].shape[0]
cntf = df[df['request_status'] == 'fail'].shape[0]

df['request_date'] = df['request_date'].apply(pd.to_datetime)
df['year_month'] =  df['request_date'].dt.to_period('M')
df['dis_per_dollar'] = (df['distance_to_travel'].astype(float))/df['monetary_cost'].astype(float)

df['month_avg'] =  df.groupby(['year_month'])['dis_per_dollar'].transform('mean')#.reset_index()

#diff = round(diff, 2)

df['diff'] = abs(df['dis_per_dollar'] - df['month_avg'])
 
resu = df.groupby('year_month')['diff'].mean().reset_index()
resu['diff'] = round(resu['diff'], 2)
resu.head()

,year_month,diff
0,2020-01,3.32
1,2020-02,1.58
2,2020-03,3.13
3,2020-04,1.19
4,2020-05,6.44


# Percentage Of Total Spend

Calculate the percentage of the total spend a customer spent on each order. Output the customer’s first name, order details, and percentage of the order cost to their total spend across all orders. 



Assume each customer has a unique first name (i.e., there is only 1 customer named Karen in the dataset) and that customers place at most only 1 order a day.



Percentages should be represented as fractions

In [136]:
# Import your libraries
import pandas as pd
path1 = '/home/kassa/Downloads/StrataScratch_files/orders.html'
path2 = '/home/kassa/Downloads/StrataScratch_files/customers.html'
orders = getdata_fromhtml(path1)
customers = getdata_fromhtml(path2)

# Start writing code
df = pd.merge(orders, customers, left_on ='cust_id', right_on = 'id', how = 'inner')
df['total_order_cost'] = df.total_order_cost.astype(float)
df['percentage'] = 100*df['total_order_cost']/df.groupby(['cust_id', 'first_name'])['total_order_cost'].transform('sum')

df = df[['first_name', 'order_details', 'percentage']]
df.head()

,first_name,order_details,percentage
0,Farida,Coat,38.461538
1,Farida,Shoes,30.769231
2,Farida,Skirt,11.538462
3,Farida,Shirts,19.230769
4,Jill,Coat,4.672897


# New Products

You are given a table of product launches by company by year. Write a query to count the net difference between the number of products companies launched in 2020 for the first time with the number of products companies launched in the previous year. Output the name of the companies and a net difference of net products released for 2020 compared to the previous year. If a company is new or had no products in 2019, then any product released in 2020 would be considered as new.


In [69]:
# Import your libraries
import pandas as pd
path = '/home/kassa/Downloads/StrataScratch_files/car_launches.html'
# Start writing code
df = getdata_fromhtml(path)
df_19 = df[df['year'] == '2019'].add_prefix('2019_')
df_19 = df_19.groupby('2019_company_name')['2019_product_name'].nunique().to_frame('nunique_prod_19').reset_index()

df_20 = df[df['year'] == '2020'].add_prefix('2020_')
df_20 = df_20.groupby('2020_company_name')['2020_product_name'].nunique().to_frame('nunique_prod_20')

merged = pd.merge(df_19, df_20, left_on = '2019_company_name', right_on = '2020_company_name', how= 'outer')
merged['diff'] = merged['nunique_prod_20'] - merged['nunique_prod_19'] 
#merged = merged[merged['diff'] >= 0]
res = merged[['2019_company_name', 'diff']]
res.sort_values('2019_company_name')

,2019_company_name,diff
0,Chevrolet,2
1,Ford,-1
2,Honda,-3
3,Jeep,1
4,Toyota,-1


# Highest Target Under Manager

Find the highest target achieved by the employee or employees who works under the manager id 13. Output the first name of the employee and target achieved. The solution should show the highest target achieved under manager_id=13 and which employee(s) achieved it.


In [63]:
import pandas as pd

path = '/home/kassa/Downloads/StrataScratch_files/salseforce_employees.html'
df = getdata_fromhtml(path)
df = df[df['manager_id'].astype(int) == 13]
df = df[df['target'] == df['target'].max() ][['first_name', 'target']]
df

,first_name,target
2,Nicky,400
3,Steve,400
4,David,400


# Risky Projects

Identify projects that are at risk for going overbudget. A project is considered to be overbudget if the cost of all employees assigned to the project is greater than the budget of the project. 

You'll need to prorate the cost of the employees to the duration of the project. For example, if the budget for a project that takes half a year to complete is $10K, then the total half-year salary of all employees assigned to the project should not exceed $10K. Salary is defined on a yearly basis, so be careful how to calculate salaries for the projects that last less or more than one year.

Output a list of projects that are overbudget with their project name, project budget, and prorated total employee expense (rounded to the next dollar amount).

# Approaches

- To calculate the expense of a project, you'll need to determine the project length in days and prorate employees salaries for respective project duration

- The budget is already given at the project level

- You're only concerned with projects that have employees and budget information so you can join all three tables by an INNER JOIN to get all the required details

- Calculate expense (sum of salaries of all employees aligned to the respective project) at project level using group by 

- Merge expense table with the combined table created in above step where project budget and duration can be found

- Select all the projects where expense is higher budget

In [137]:
# Import your libraries
import pandas as pd
import datetime as dt
import numpy as np

# Start writing code
path1 = '/home/kassa/Downloads/StrataScratch_files/linkedin_projects.html'
path2 = '/home/kassa/Downloads/StrataScratch_files/linkedin_emp_projects.html'
path3 = '/home/kassa/Downloads/StrataScratch_files/linkedin_employees.html'

linkedin_projects = getdata_fromhtml(path1)
linkedin_emp_projects = getdata_fromhtml(path2)
linkedin_employees = getdata_fromhtml(path3)

merged1 = pd.merge(linkedin_projects, linkedin_emp_projects, left_on = 'id', right_on = 'project_id', how= 'inner')
merged = pd.merge(merged1, linkedin_employees, left_on = 'emp_id', right_on = 'id', how = 'inner')

merged[['salary', 'budget']] = merged[['salary', 'budget']].astype(float)
merged[['start_date', 'end_date']] = merged[['start_date', 'end_date']].apply(pd.to_datetime)

merged['salary_during'] = merged['salary'] * (merged['end_date'] - merged['start_date']).dt.days/365 
merged
merged = merged.groupby(['title', 'budget'])['salary_during'].sum().reset_index()
merged['salary_total'] = merged['salary_during'].apply(np.ceil)
 
merged['diff'] = merged['budget'] - merged['salary_total']
risky_projects = merged[merged['diff'] < 0]
result = risky_projects[['title', 'budget', 'salary_total']]
result.head()

,title,budget,salary_total
0,Project1,29498.0,36293.0
2,Project11,11705.0,31606.0
3,Project12,10468.0,62843.0
5,Project14,30014.0,36774.0
7,Project16,19922.0,21875.0


# Highest Energy Consumption

Find the date with the highest total energy consumption from the Meta/Facebook data centers. Output the date along with the total energy consumption across all data centers.

# Approaches

- Combine together all three tables using concat()

- Sum the consumption values by date 

- Find the max consumption value and then filter on the max value to find the dates of max consumption

In [72]:
import pandas as pd

path1 = '/home/kassa/Downloads/StrataScratch_files/fb_eu_energy.html'
path2 = '/home/kassa/Downloads/StrataScratch_files/fb_asia_energy.html'
path3 = '/home/kassa/Downloads/StrataScratch_files/fb_na_energy.html'

fb_eu_energy = getdata_fromhtml(path1)
fb_asia_energy = getdata_fromhtml(path2)
fb_na_energy = getdata_fromhtml(path3)

df = pd.concat([fb_eu_energy, fb_asia_energy, fb_na_energy])
df['consumption'] = df['consumption'].astype(float)
df = df.groupby('date')['consumption'].sum().to_frame('total').reset_index()

df[df['total'] == df['total'].max()][['date', 'total']]

,date,total
5,2020-01-06,1250.0
6,2020-01-07,1250.0


In [7]:
round(abs(0.131-0.176), 2)

0.04

In [8]:
df = pd.DataFrame({"Letters": ["a", "b", "c"], "Numbers": [1, 2, 3]})
df.iloc[[0,2],:]

,Letters,Numbers
0,a,1
2,c,3


# Finding User Purchases

Write a query that'll identify returning active users. A returning active user is a user that has made a second purchase within 7 days of any other of their purchases. Output a list of user_ids of these returning active users.

# Approaches

- For each row, join rows respresenting following events in the datasets

- For each pair of events calculate the difference in days

- Filter events that have less then 8 days break

In [7]:
# Import your libraries
import pandas as pd
import datetime as dt

path = '/home/kassa/Downloads/StrataScratch_files/amazon_transaction.html'
# Start writing code
df = getdata_fromhtml(path) 
df['created_at'] = df['created_at'].apply(pd.to_datetime)
#df['user_id'] = df['user_id'].astype(int)
df = df.sort_values(['user_id', 'created_at'], ascending = [True, True]) 
df['pre_date'] = df.groupby('user_id')['created_at'].shift().apply(pd.to_datetime)
print(df['pre_date'].dt.day.shape[0])
df_n = df[df['created_at'].dt.day - df['pre_date'].dt.day <= 7]['user_id'].unique()  



100


array(['100', '103', '105', '109', '110', '111', '112', '114', '117',
       '120', '122', '128', '129', '130', '131', '133', '141', '143',
       '150'], dtype=object)

# Finding Updated Records

We have a table with employees and their salaries, however, some of the records are old and contain outdated salary information. Find the current salary of each employee assuming that salaries increase each year. Output their id, first name, last name, department ID, and current salary. Order your list by employee ID in ascending order.

# Approaches

- Ideally this dataset should consists of unique records of employees for only current year
- Due to this ETL error multiple years of data can be found for some employees
- Until now at every compensation revision cycle, all employees have received a salary increase so you can assume that the highest salary is the employee's current salary. Use a max() function to find the highest salary for each employee.
- The output should be all the details of all the employees with correct salary

In [138]:
import pandas as pd

path = '/home/kassa/Downloads/StrataScratch_files/ms_employee_salary.html'

df = getdata_fromhtml(path)

new_df = df.groupby(['id', 'first_name', 'last_name', 'department_id'])['salary'].max().reset_index()
result = new_df.sort_values('id')
result.head()

,id,first_name,last_name,department_id,salary
0,1,Todd,Wilson,1006,110000
1,10,Sean,Crawford,1006,190000
2,11,Kevin,Townsend,1002,166861
3,12,Joshua,Johnson,1004,123082
4,13,Julie,Sanchez,1001,210000


# Number of Streets Per Zip Code

Find the number of different street names for each postal code, for the given business dataset. For simplicity, just count the first part of the name if the street name has multiple words. 


For example, East Broadway can be counted as East. East Main and East Broadly may be counted both as East, which is fine for this question. 


Counting street names should also be case insensitive, meaning FOLSOM should be counted the same as Folsom. Lastly, consider that some street names have different structures. For example, Pier 39 is not the same as 39 Pier, and E Broadway is not the same as East Broadway, so your solution should count both situations correctly.


Output the result along with the corresponding postal code. Order the result based on the number of streets in descending order and based on the postal code in ascending order.

----------------------------------------------------------

- Consider situation where the address has different structure: with number on the first place and with street name on the first place


- notnull() can be used to determine the non-missing values in the dataframe


- Split the address into words by converting the object to str first, using astype(str), then applying str.split()


- Use str.lower() to convert strings to lower case


- Use nunique() to get the number of distinct observations per group or specified axis then convert the resulting object to a dataframe using to_frame('column_name')


- Use sort_values(column_name, order) to sort along a specified column; Set order to False to display the printed values in descending order

In [12]:
# Import your libraries
import pandas as pd

path = '/home/kassa/Downloads/StrataScratch_files/sf_restaurant_health_violations.html'


df = getdata_fromhtml(path)

def split_is_numeric(x):
    if str(x.split(' ')[0]).isdigit() == True:
        return  x.split(' ')[1].lower()
    else:
        return x.split(' ')[0].lower()

df['add_new'] = df['business_address'].apply(split_is_numeric) 

resu = df.groupby('business_postal_code')['add_new'].nunique().to_frame('cnt_street').sort_values('cnt_street').reset_index()
resu.head()

,business_postal_code,cnt_street
0,94131,1
1,94127,2
2,94116,2
3,94124,3
4,94123,3


# Ranking Most Active Guests

Rank guests based on the number of messages they've exchanged with the hosts. Guests with the same number of messages as other guests should have the same rank. Do not skip rankings if the preceding rankings are identical.


Output the rank, guest id, and number of total messages they've sent. Order by the highest number of total messages first.

------------------------

- You're going to want to use a ranking function to find the ranks and set the method to 'dense' so that the rankings don't skip.

In [71]:
import pandas as pd

# Start writing code
path = '/home/kassa/Downloads/StrataScratch_files/airbnb_contacts.html'
df = getdata_fromhtml(path)
df = df.groupby('id_guest')['n_messages'].sum().reset_index().sort_values('n_messages', ascending = False)

df['rank'] = df.n_messages.rank(method = 'dense', ascending = False).astype('int')
result = df[['rank', 'id_guest', 'n_messages']]
result.head()

,rank,id_guest,n_messages
29,1,5ed478e0-eae9-4537-826a-04ec779c1fd2,9
4,1,125242de-91b4-43bd-a926-75ae4b7a9322,9
16,1,29ef2346-5fda-45d7-b4ed-eb34967a62b3,9
46,1,845e3c1d-6c06-4ef2-ade5-7eecefb56fbe,9
77,2,d418a1ab-b181-40a7-90fe-7216e40dc354,8


# Income By Title and Gender

Find the average total compensation based on employee titles and gender. Total compensation is calculated by adding both the salary and bonus of each employee. However, not every employee receives a bonus so disregard employees without bonuses in your calculation. 


Output the employee title, gender (i.e., sex), along with the average total compensation.

------------------

- Take a look at the data structure. Worker_ref_id in table sf_bonus is not primary key. This is important information when joining two tables.

In [18]:
import pandas as pd

# Start writing code
path1 = '/home/kassa/Downloads/StrataScratch_files/sf_employee.html'
path2 = '/home/kassa/Downloads/StrataScratch_files/sf_bonous.html'
sf_bonus = getdata_fromhtml(path2)
sf_employee = getdata_fromhtml(path1)
sf_bonus['bonus'] = sf_bonus['bonus'].astype(float)
sf_bonus['worker_ref_id'] = sf_bonus['worker_ref_id'].astype(int)
sf_employee['id'] = sf_employee['id'].astype(int)

sf_summery = sf_bonus.groupby(['worker_ref_id'])['bonus'].sum().reset_index()

df = pd.merge(sf_employee, sf_summery, left_on = 'id', right_on = 'worker_ref_id', how = 'right')
df['salary'] = df['salary'].astype(float)

df['compensation'] = df['salary'] + df['bonus']

df = df.groupby(['employee_title', 'sex'])['compensation'].mean().reset_index()
df

,employee_title,sex,compensation
0,Manager,F,209500.0
1,Sales,M,7500.0
2,Senior Sales,M,6000.0


# Find all wineries  

Find all wineries which produce wines by possessing aromas of plum, cherry, rose, or hazelnut.
Output unique winery values only.


# Approaches

- Use the ILIKE operator on the description column for string patterns '%plum%', '%cherry%', '%rose%', and '%hazelnut%'.

- Use the OR operator to combine all records that satisfy any of the aforementioned string patterns.

- Use the WHERE clause to apply conditions to the dataset.

In [21]:
# Import your libraries
import pandas as pd

# Start writing code

 
df = getdata_fromhtml('/home/kassa/Downloads/StrataScratch_files/winemag.html')
df = df[df['description'].str.lower().str.contains('plum|cherry|rose|hazelnut')]

res = df['winery'].drop_duplicates()
res.head()

3     Boudreaux Cellars
7             Goldeneye
9            Pine Ridge
14              HÌ¦pler
15         Bella Piazza
Name: winery, dtype: object

# Counting Instances in Text

Find the number of times the words 'bull' and 'bear' occur in the contents. We're counting the number of times the words occur so words like 'bullish' should not be included in our count.


Output the word 'bull' and 'bear' along with the corresponding number of occurrences.

# Approache

- Create a new dataframe using pd.DataFrame with columns = ['word','netry'] 


- Where in word column contains the words we want to look for 'bull', 'bear' and netry with the count of each word using str.count('pattern')


- Use sum() to get the total occurences per word

In [24]:
# Import your libraries
import pandas as pd

# Start writing code
df = getdata_fromhtml('/home/kassa/Downloads/StrataScratch_files/google_file.html') 
data = {'word' : ['bull', 'bear'], 'netry' : [df.contents.str.count(' bull ').sum(), df.contents.str.contains(' bear ').sum()]}

res = pd.DataFrame(data, columns = ['word', 'netry'])
res 

,word,netry
0,bull,3
1,bear,2


# Count the number of user events performed by MacBookPro users

Count the number of user events performed by MacBookPro users.
Output the result along with the event name.


Sort the result based on the event count in the descending order.

# Approache

- Select specific column/s from dataframe using [column_anme] then select rows with values equal to ‘==’ macbook pro

- Use .groupby(column_name) to group the dataframe about the specifed column

- Use size() to get the number of elements in the specified column

- Use sort_values(column_name, order) to sort along a specified column; Set order to False to display the printed values in descending order 

- Use .rename(columns = {'old_name':'new_name'}) to rename specific columns

In [139]:
# Import your libraries
import pandas as pd

# Start writing code
df = getdata_fromhtml('/home/kassa/Downloads/StrataScratch_files/playbook_events.html')

df = df[df['device'] == 'macbook pro']
res = df.groupby('event_name').size().to_frame('n_userevents').reset_index()

res.head()

,event_name,n_userevents
0,home_page,8
1,like_message,3
2,login,3
3,search_autocomplete,3
4,send_message,1


# Find libraries who haven't provided the email address in 2016 but their notice preference definition is set to email

Find libraries who haven't provided the email address in 2016 but their notice preference definition is set to email.


Output the library code.

# Approaches

- Select specific column/s from dataframe using [column_name] then select rows with values equal to ‘==’ email,False, and 2016


- Use AND '&' to check if conditions are satisfied (True)


- Use [ [ column_name/s] ] to return a specified column of the dataframe


In [51]:
# Import your libraries
import pandas as pd

# Start writing code
df = getdata_fromhtml('/home/kassa/Downloads/StrataScratch_files/library_usage.html') 
df = df.replace(r'^\s*$', 'NaN', regex = True)
df = df[(df['circulation_active_year'] == 2016) &  (df['notice_preference_definition'].astype(str) == 'email') &  (df['provided_email_address'].astype(str) == False)]

resu = df['home_library_code'].astype(str)
resu
 

Series([], Name: home_library_code, dtype: object)

# Find matching hosts and guests in a way that they are both of the same gender and nationality


Find matching hosts and guests pairs in a way that they are both of the same gender and nationality.
Output the host id and the guest id of matched pair.

# Approaches

- Use pd.merge(dataframe1, dataframe2, on = common_table_keys) to perform inner join on the dataframes


- Use [ [ column_name/s] ] to return a specified column of the dataframe


In [54]:
# Import your libraries
import pandas as pd

df1 = getdata_fromhtml('/home/kassa/Downloads/StrataScratch_files/airbmb_host.html')
df2 = getdata_fromhtml('/home/kassa/Downloads/StrataScratch_files/airbnb_guests.html')
merged = pd.merge(df1, df2, left_on = ['nationality', 'gender'], right_on = ['nationality', 'gender'], how = 'inner')
 
result = merged[['host_id', 'guest_id']]
result.head()

,host_id,guest_id
0,0,9
1,1,5
2,2,1
3,3,7
4,4,0


# Top Ranked Songs

Find songs that have ranked in the top position. Output the track name and the number of times it ranked at the top. Sort your records by the number of times the song was in the top position in descending order.

# Approaches

- Select specific column/s from dataframe using [column_name] then select rows with values equal to ‘==’ 1

- Use .groupby(column_name) to group the dataframe about the specifed column then size() to get the number of elements or count per group

- Convert the resulting object to a dataframe using to_frame()

- Use sort_values(column_name, order) to sort along a specified column; Set order to False to display the printed values in descending order


In [58]:
# Import your libraries
import pandas as pd

# Start writing code
df = getdata_fromhtml('/home/kassa/Downloads/StrataScratch_files/spotify_worldwide_daily_song_ranking.html')

df = df[df['position'].astype(int) == 1]
resu = df.groupby('trackname').size().to_frame('n_topposition').reset_index().sort_values('n_topposition', ascending = False)
resu

,trackname,n_topposition
1,HUMBLE.,7
0,Bad and Boujee (feat. Lil Uzi Vert),1
2,Look What You Made Me Do,1


# Churro Activity Date


Find the activity date and the pe_description of facilities with the name 'STREET CHURROS' and with a score of less than 95 points.

# Approaches


- Select specific column/s from dataframe using [column_name] then select rows with values equal to specified operators ( ==, <, etc)

- Use AND '&' to check if both conditions are satisfied (True)

- Use [ [ column_name/s] ] to return a specified column of the dataframe

- Use to_datetime() to convert timestamp to date format

In [62]:
# Import your libraries
import pandas as pd
import datetime as dt

# Start writing code
df = getdata_fromhtml('/home/kassa/Downloads/StrataScratch_files/los_angeles_restaurant_health_inspections.html')
df = df[(df['facility_name'].str.lower() == 'street churros') & (df['score'].astype(float) < 95)]
df['activity_date'] = pd.to_datetime(df['activity_date'], format = '%Y-%m-%d' ).dt.strftime('%Y-%m-%d')

result = df[['activity_date', 'pe_description']]
result

,activity_date,pe_description
161,2017-12-29,RESTAURANT (0-30) SEATS LOW RISK
162,2016-12-01,RESTAURANT (0-30) SEATS LOW RISK
163,2016-06-16,RESTAURANT (0-30) SEATS LOW RISK


In [61]:
x = ['ab', 'bb', 'cc', 'abay', 'kassa', 'abbe']
a = list(map(lambda x: len(x) == 2, x))
 
a

[True, True, True, False, False, False]

# Toptal

# Task 1

You would like to find the sentance containing the largest number of words in some given text. The text is specified a strong 
S consisting of N characters: letters, spaces, dots, question marks, and exclamation marks. 

The text can be divided into sentances by spliting it at dots, question marks, and exclamation marks. A sentance can be divided into words by spliting 
it at spaces. A setance without words is valid, but a valid word must contain at least one letter. 


In [7]:
import re
test='hello! How are you? I am fine, thank you. And you?'

l = re.split('[?] |[!] |[.] |\*|\n',test)
x = [len(x.split(' ')) for x in l]

n_max = x.index(max(x))
print(l[n_max])
print(l)
print(x)
n_max

I am fine, thank you
['hello', 'How are you', 'I am fine, thank you', 'And you?']
[1, 3, 5, 2]


2

In [6]:
#Task 2

# extract the month
mon = [1,1,2,3,4,5,5,5,6,7,7]
pay = [1,2,1,-2, 4, -3,-3,-4, 1, 2,2]

z = [(mon[ind], val) for ind, val in enumerate(pay) if val < 0]
print(pair)
for i in range(1, len(m)-1):
    if (z[i-1][0] == z[i][0] == z[i+1][0]) & (z[i-1][0] + z[i][0] + z[i+1][0]):
        

[(3, -2), (5, -3), (5, -3), (5, -4)]


# Top Percentile Fraud

ABC Corp is a mid-sized insurer in the US and in the recent past their fraudulent claims have increased significantly for their personal auto insurance portfolio. They have developed a ML based predictive model to identify propensity of fraudulent claims. Now, they assign highly experienced claim adjusters for top 5 percentile of claims identified by the model.

Your objective is to identify the top 5 percentile of claims from each state. Your output should be policy number, state, claim cost, and fraud score.

# Approaches

- Use a window function that will rank the records by percentile to 100.

- Partition by/group by the state

- Limit to only the top 5 percent of top fraud scores

In [67]:
import pandas as pd

# Start writing code
df = getdata_fromhtml('/home/kassa/Downloads/StrataScratch_files/fraud_score.html')
df['fraud_score'] = df['fraud_score'].astype(float)
df = df.assign(
    percentile=(100 * df.groupby('state')['fraud_score']
                .rank(ascending=False, pct=True, method='first')).astype(int))
df = df.query('percentile <= 5')

result = df.drop(['percentile'], axis = 1)
result.head()

,policy_num,state,claim_cost,fraud_score
15,ABCD1016,CA,1639,0.964
26,ABCD1027,CA,2663,0.988
68,ABCD1069,CA,1426,0.948
78,ABCD1079,CA,4224,0.963
80,ABCD1081,CA,1080,0.951


# Arizona, California, and Hawaii Employees
Find employees from Arizona, California, and Hawaii while making sure to output all employees from each city. Output column headers should be Arizona, California, and Hawaii. Data for all cities must be ordered on the first name.

Assume unequal number of employees per city.

# Approach

- Filter out rows by selecting specific column/s from dataframe using [column_name] then select rows under city that is equal to Arizona, California and Hawaii. Do this by defining a list containing these cities and isin()

- Pivot the dataframe using pd.pivot_table with city as column, first_name as values.

- To return string values, agg func will be set as lambda x: ‘, ‘.join(x) But this will return a boolean. In the aggfunc field you need to use that small loop to return every specific value: join(str(v) for v in x)

- Use along with it dropna() to remove null values from each column

- Clean resulting multilevel indexes by applying reset_index()

- Use [ [ column_name/s] ] to return a specified column of the dataframe

In [72]:
import pandas as pd

# Start writing code
df = pd.read_csv('employee.csv')
city_list = ['Arizona', 'California', 'Hawaii']

df = df[df['city'].isin(city_list)]

new = pd.pivot_table(df,columns='city', values = 'first_name', aggfunc = lambda x: ' '.join(str(x)))
new

city,Arizona,California,Hawaii
first_name,1 K a t t y \n 1 5 ...,0 M a x \n 2 6 A l l e n...,1 0 J a c k \n 1 1 ...


# Salaries Differences

Write a query that calculates the difference between the highest salaries found in the marketing and engineering departments. Output just the absolute difference in salaries

# Approaches

- JOIN the department table with employee table to get a list of employees, salaries, and department

- GROUP BY max salary by department in two different data frames

- Extract difference of highest salaries between two departments in a different data frame

In [4]:
import pandas as pd

# Start writing code
db_employee = getdata_fromhtml('/home/kassa/Downloads/StrataScratch_files/db_employee.html')
db_dept = getdata_fromhtml('/home/kassa/Downloads/StrataScratch_files/db_dept.html')


df = pd.merge(db_employee, db_dept, how = 'left', left_on = ['department_id'], right_on = ['id'])
# Select the enginering department 
df['salary'] = df['salary'].astype(float)

df_eng = df[df['department'] == 'engineering']
df_eng_max = df_eng.groupby('department')['salary'].max().reset_index(name = 'eng_salary')

df_mkt = df[df['department'] == 'marketing']
df_mkt_max = df_mkt.groupby('department')['salary'].max().reset_index(name = 'mkt_salary')
max_dif = abs(pd.DataFrame(df_eng_max['eng_salary'] - df_mkt_max['mkt_salary']))
max_dif.columns = ['salary_difference']

max_dif

,salary_difference
0,2400.0


# Acceptance Rate By Date

What is the overall friend acceptance rate by date? Your output should have the rate of acceptances by the date the request was sent. Order by the earliest date to latest.

Assume that each friend request starts by a user sending (i.e., user_id_sender) a friend request to another user (i.e., user_id_receiver) that's logged in the table with action = 'sent'. If the request is accepted, the table logs action = 'accepted'. If the request is not accepted, no record of action = 'accepted' is logged.

# Approaches

- Assume that each friend request starts by a user sending a friend request that's logged in the table with action = 'sent'

- When a friend request is accepted, a notification is sent to the user that sent the original friend request and action = 'accepted'. This also means that the original user_id_sender is now logged as user_id_accepted and vice versa for the original receiver. You'll need to filter your tables using this logic.

- Create two subsets, one with 'accepted' and other with 'sent' request

- Merge both subsets with sender and receiver key

- Check if sent request is accepted or not across the dates

- Count overall sent request by date and respective acceptances

- Calculate acceptance rate

In [6]:
import pandas as pd

# Start writing code
df = getdata_fromhtml('/home/kassa/Downloads/StrataScratch_files/fb_friend_request.html')

df_sent = df[df.action == 'sent']
df_accepted = df[df.action == 'accepted']

df = pd.merge(df_sent, df_accepted, how = 'left', left_on = ['user_id_sender', 'user_id_receiver'], right_on = ['user_id_sender', 'user_id_receiver'])
accepted_count = df.groupby('date_x').count().reset_index()

accepted_count['Acceptance_rate'] = accepted_count['action_y']/accepted_count['action_x']
result = accepted_count[['date_x', 'Acceptance_rate']]
result.head()

,date_x,Acceptance_rate
0,2020-01-04,0.750000
1,2020-01-06,0.666667


# Popularity Percentage

Find the popularity percentage for each user on Meta/Facebook. The popularity percentage is defined as the total number of friends the user has divided by the total number of users on the platform, then converted into a percentage by multiplying by 100.

Output each user along with their popularity percentage. Order records in ascending order by user id.

The 'user1' and 'user2' column are pairs of friends.

# Approaches 

- You’ll need to create two subqueries or CTEs to calculate total unique friends-pair and total number of friend per user

- To calculate total unique friends-pair, UNION the user1 and user2 columns. This will de-duplicate any users that are repeated in the column. You can ensure uniqueness by adding a DISTINCT. To calculate the total count of unique friends-pair, perform a count(*) on the subquery.

- To calculate the total number of friends per user, create one column that contains all users and a second column that contains all their friends. You can accomplish this by user1, user2 UNION user2, user1.

- JOIN the two subqueries together so that you can calculate the percentage of friends over total users in the platform. You’ll need to JOIN using a 1=1 relation as the key. The resulting table will be a list of users, their friends, and total number of users on the platform.

- Lastly, implement the percentage popularity formula by counting the number of friends per user and dividing by the total number of users on the platform.

In [9]:
import pandas as pd
import numpy as np

# Start writing code
df = getdata_fromhtml('/home/kassa/Downloads/StrataScratch_files/facebook_friends.html')

arr = len(np.unique(np.concatenate([df.user1.values, df.user2.values])))
new_df = df.rename(columns = {'user1': 'user2', 'user2': 'user1'})
final = pd.concat([df, new_df], sort = False).drop_duplicates()
res = final.groupby('user1').size().to_frame('count').reset_index()
res['popularity_percentage'] = (res['count']/arr)*100
res.head()

,user1,count,popularity_percentage
0,1,5,55.555556
1,2,3,33.333333
2,3,3,33.333333
3,4,1,11.111111
4,5,1,11.111111


# Users By Average Session Time
Calculate each user's average session time. A session is defined as the time difference between a page_load and page_exit. For simplicity, assume a user has only 1 session per day and if there are multiple of the same events on that day, consider only the latest page_load and earliest page_exit. Output the user_id and their average session time.

# Approaches

-  Select from the table only events that are crucial for the analysis: page load and exit

- Consider different cases that appear in the data set

- For users loading the page multiple times, assume that there is a unique page visit within the hour.

In [7]:
import pandas as pd

# Start writing code
df = getdata_fromhtml('/home/kassa/Downloads/StrataScratch_files/facebook_web_logs.html')
# a page load  | 
df['timestamp'], df['user_id'] = df['timestamp'].apply(pd.to_datetime), df['user_id'].astype(int)

df_load = df[(df['action'] == 'page_load')]
df_load['day'] = df['timestamp'].dt.day
df_load = df_load.groupby(['user_id', 'day'])['timestamp'].max().reset_index()

# an exit 
df_exit = df[(df['action'] == 'page_exit')]
df_exit['day'] = df_exit['timestamp'].dt.day
#df_exit['day'] = df_exit['day'].astype(int)

df_exit = df_exit.groupby(['user_id', 'day'])['timestamp'].min().reset_index()

df_new = pd.merge(df_load, df_exit, on = ['user_id','day'], suffixes=['_page_load', '_page_exit'])

df_new['session_time'] = (df_new.timestamp_page_exit - df_new.timestamp_page_load).dt.seconds
df_new = df_new.groupby('user_id')['session_time'].mean().reset_index()
df_new

/usr/lib/python3/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/lib/python3/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,user_id,session_time
0,0,1883.5
1,1,35.0


# Highest Cost Orders (Medium)
Find the customer with the highest daily total order cost between 2019-02-01 to 2019-05-01. If customer had more than one order on a certain day, sum the order costs on daily basis. Output their first name, total cost of their items, and the date.

For simplicity, you can assume that every first name in the dataset is unique.

# Approaches

- Use pd.merge(dataframe1, dataframe2, on = common_table_keys, how = 'left') to perform join on the dataframes

- Convert column to datetime format using pd.to_datetime then filter in between dates using .between(first_day, last_day)

- Use groupby() to group along a specific column then use max() to get the maximum value per group

- Convert the resulting object to a dataframe using to_frame()

- Filter resulting df by maximum value

In [11]:
import pandas as pd
customers = getdata_fromhtml('/home/kassa/Downloads/StrataScratch_files/customers.html')
orders = getdata_fromhtml('/home/kassa/Downloads/StrataScratch_files/orders.html')
orders['order_date'], orders['cust_id'] = orders['order_date'].apply(pd.to_datetime), orders['cust_id'].astype(int)
customers['id'] = customers['id'].astype(int)

daily_orders = orders[(orders['order_date'] >= '2019-02-01') & (orders['order_date']<= '2019-05-01')].groupby(['cust_id', 'order_date'])['total_order_cost'].sum().reset_index()


merged = pd.merge(daily_orders, customers[['id', 'first_name']], left_on = 'cust_id', right_on = 'id', how = 'inner' )
merged['order_date'] = merged['order_date'].dt.date
merged[merged.total_order_cost == merged.total_order_cost.max()][['first_name', 'order_date', 'total_order_cost']]

,first_name,order_date,total_order_cost
0,Farida,2019-03-01,80
4,William,2019-02-01,80
5,Henry,2019-02-01,80
7,Jill,2019-03-10,80
15,Mia,2019-03-01,80


# Monthly Percentage Difference
- Hard

Given a table of purchases by date, calculate the month-over-month percentage change in revenue. The output should include the year-month date (YYYY-MM) and percentage change, rounded to the 2nd decimal point, and sorted from the beginning of the year to the end of the year.

The percentage change column will be populated from the 2nd month forward and can be calculated as ((this month's revenue - last month's revenue) / last month's revenue)*100.

# Approaches

- Make sure 'created_at' date is in datetime format

- Derive a 'year_month' column from 'created_at'

- Sort values in ascending order by date

- Use the shift() function to get the previous month's revenue value then calculate the revenue difference between the current month and the previous month

- Calculate the difference in month-over-month revenue 

- Calculate percentage growth month-over-month revenue

- Replace first row's NaN values to blank using fillna()

In [16]:
import pandas as pd
df = getdata_fromhtml('/home/kassa/Downloads/StrataScratch_files/sf_transaction.html')

df['created_at'], df['value'] = df['created_at'].apply(pd.to_datetime), df['value'].astype(int)
#df['MOnths'] = month
df['year_month'] = pd.to_datetime(df['created_at']).dt.to_period('M')

df = df.groupby('year_month')['value'].sum().reset_index(name ='monthly_revenue').sort_values('year_month')
df['prev_month'] = df['monthly_revenue'].shift(1)
per = (df['monthly_revenue']-df['prev_month'])/df['prev_month']

df['perce_diff'] = round(per*100, 2)

fina_resu = df[['year_month', 'perce_diff']].fillna('')
fina_resu.head()

,year_month,perce_diff
0,2019-01,
1,2019-02,-28.56
2,2019-03,23.35
3,2019-04,-13.84
4,2019-05,13.49


# Premium vs Freemium
  - Hard

Find the total number of downloads for paying and non-paying users by date. Include only records where non-paying customers have more downloads than paying customers. The output should be sorted by earliest date first and contain 3 columns date, non-paying downloads, paying downloads.

# Approaches

- ms_user_dimension will help link the number of downloads with whether an account is paying or non-paying

- Join ms_user_dimension, ms_acc_dimension, ms_download facts using LEFT JOIN

- Create columns for total downloads for non paying customers and paying customers using PIVOT

- The output should contain 3 columns date, non paying downloads and paying downloads and output shoud be sorted by date in ascending order.

In [18]:
import pandas as pd

# Start writing code
ms_user_dimension = getdata_fromhtml('/home/kassa/Downloads/StrataScratch_files/ms_user_dimension.html')
ms_acc_dimension = getdata_fromhtml('/home/kassa/Downloads/StrataScratch_files/ms_acc_dimension.html')
ms_download_facts = getdata_fromhtml('/home/kassa/Downloads/StrataScratch_files/ms_download_facts.html')

merge1 = pd.merge(ms_user_dimension, ms_acc_dimension, left_on = 'acc_id', right_on = 'acc_id', how= 'inner')

merged = pd.merge(merge1, ms_download_facts, on = 'user_id', how = 'inner')

merged = merged.groupby(['user_id', 'date', 'paying_customer'])['downloads'].sum().reset_index() 
x=merged.pivot_table(index=['date'],columns=['paying_customer'],values=['downloads'],aggfunc='sum')
df = pd.DataFrame(x.to_records())
df.columns = df.columns.str.replace('[()]', '').str.replace('[,]', '').str.replace('downloads', '').str.replace("[' ']", "")
df[df.no > df.yes][['date', 'no', 'yes']].sort_values('date').head()

#ms_user_dimension

,date,no,yes
0,2020-08-15,56,36622
2,2020-08-17,816244785,81
3,2020-08-18,523,52
4,2020-08-19,9103,67
5,2020-08-20,5602,08776


# Top 5 States With 5 Star Businesses
   - Medium

Find the top 5 states with the most 5 star businesses. Output the state name along with the number of 5-star businesses and order records by the number of 5-star businesses in descending order. In case there are ties in the number of businesses, return all the unique states. If two states have the same result, sort them in alphabetical order.

# Approaches

- Select specific column/s from dataframe using [column_name] then select rows with values equal to ‘==’ 5

- Use .groupby(column_name) to group the dataframe about the specified column then use count() to get the number of values per group

- Convert the resulting object to a dataframe using to_frame()

- Use sort_values(column_name, order) to sort along a specified column; Set order to False to display the printed values in descending order 

- Use rank function to find top 5 states

In [22]:
import pandas as pd

# Start writing code
df = getdata_fromhtml('/home/kassa/Downloads/StrataScratch_files/yelp_business.html')
df['stars'] = df['stars'].astype(float)
df = df[df['stars'] == 5]
df = df.groupby('state')['stars'].count().to_frame('no_five_stared_business').sort_values(['no_five_stared_business', 'state'], ascending = [False, True]).reset_index()

df['rank'] = df['no_five_stared_business'].rank(method = 'min', ascending = False)
df = df[df['rank'] <= 5][['state', 'no_five_stared_business']]
df.head()

,state,no_five_stared_business
0,AZ,10
1,ON,5
2,NV,4
3,IL,3
4,OH,3


# Marketing Campaign Success [Advanced]
  - Hard

You have a table of in-app purchases by user. Users that make their first in-app purchase are placed in a marketing campaign where they see call-to-actions for more in-app purchases. Find the number of users that made additional in-app purchases due to the success of the marketing campaign.

The marketing campaign doesn't start until one day after the initial in-app purchase so users that make multiple purchases on the same day do not count, nor do we count users that make only the same purchases over time.- First, find users that are not considered part of the marketing campaign due to not meeting the eligibility criteria of the campaign.

# Approaches
- Remove users that have made multiple purchases across multiple dates but only purchased the same products that were originally purchased during their first purchase. You can identify these users by concating user_id with product_id and grouping by date of purchase. Then remove the users that purchased the same product across multiple dates. 

- Remove users that only purchased products once by counting distinct/unique dates of purchases.

- Remove users that only purchased the same products





In [30]:
import pandas as pd

# Start writing code
df = getdata_fromhtml('/home/kassa/Downloads/StrataScratch_files/marketing_campaign.html')
df[['user_id', 'product_id']], df['created_at'] = df[['user_id', 'product_id']].astype(int), df['created_at'].apply(pd.to_datetime)

users_impacted_by_campaign = df.groupby('user_id')['created_at', 'product_id'].nunique().add_prefix(
    'cnt_').reset_index()
#print(users_impacted_by_campaign)
    
users_impacted_by_campaign = users_impacted_by_campaign[
    (users_impacted_by_campaign["cnt_created_at"] > 1) & (users_impacted_by_campaign["cnt_product_id"] > 1)]

first_order = df[df['created_at'] == df['created_at']]
 
result = df[(df['user_id'].isin(users_impacted_by_campaign['user_id'])) & (
    ~df['product_id'].isin(first_order['product_id']))]['user_id'].nunique()

result

/usr/lib/python3/dist-packages/ipykernel_launcher.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  import sys


0

# Customer Details
- Easy

Find the details of each customer regardless of whether the customer made an order. Output the customer's first name, last name, and the city along with the order details.

You may have duplicate rows in your results due to a customer ordering several of the same items. Sort records based on the customer's first name and the order details in ascending order.

# Approaches

- Perform Left Join on customers and orders dataframe by using pd.merge(dataframe1, dataframe2, on = common_table_keys, how = 'left')

- Use [ [ column_name/s] ] to return a specified column of the dataframe then sort along the specified column using sort_values()

In [34]:
import pandas as pd

# Start writing code
customers = getdata_fromhtml('/home/kassa/Downloads/StrataScratch_files/customers.html')
orders = getdata_fromhtml('/home/kassa/Downloads/StrataScratch_files/orders.html')

df = pd.merge(customers, orders, left_on = 'id', right_on = 'cust_id', how='left')

df_new = df[['first_name', 'last_name', 'city','order_details']].sort_values(['first_name', 'order_details']) 
df_new.head()

,first_name,last_name,city,order_details
12,Emma,Isaac,Miami,NaN
24,Eva,Lucas,Arizona,Coat
22,Eva,Lucas,Arizona,Shirts
23,Eva,Lucas,Arizona,Slipper
28,Farida,Joseph,San Francisco,Coat


# Number Of Bathrooms And Bedrooms
- Easy

Find the average number of bathrooms and bedrooms for each city’s property types. Output the result along with the city name and the property type.

# Approach

- Use .groupby(column1, column2) to group the dataframe about the specifed columns then use [column1, column2].mean() function to get the mean of values of each column

In [37]:
import pandas as pd

# Start writing code
df = getdata_fromhtml('/home/kassa/Downloads/StrataScratch_files/airbnb_search_details.html')

df[['bathrooms', 'bedrooms']] = df[['bathrooms', 'bedrooms']].astype(int)

df_m = df.groupby(['city', 'property_type'])['bathrooms', 'bedrooms'].mean().reset_index().rename(index = str, columns={'bedrooms': 'n_bedrooms_avg', 'bathrooms':'n_bathrooms_avg'})

df_m.head()

/usr/lib/python3/dist-packages/ipykernel_launcher.py:8: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,city,property_type,n_bathrooms_avg,n_bedrooms_avg
0,Boston,Apartment,1.0,1.0
1,Boston,Condominium,2.0,2.0
2,Boston,House,1.0,1.0
3,Chicago,Apartment,2.0,2.0
4,Chicago,Condominium,2.0,2.0


# Customer Revenue In March
- Medium 

Calculate the total revenue from each customer in March 2019. 
Output the revenue along with the customer id and sort the results based on the revenue in descending order.

# Appeoaches

- Convert the order_date to datetime format using pd.to_datetime

- Filter out rows under the month of March by extracting the month from the datetime using dt.month

- Create a new column that contains the quotient of order_quantitye and order_cost

- Use .groupby(column_name) to group the dataframe about the specifed column and use sum() to get the total value per group

- Convert the resulting object to a dataframe using to_frame()

- Use sort_values(column_name, order) to sort along a specified column; Set order to False to display the printed values in descending order


In [42]:
import pandas as pd
import datetime as dt


orders = getdata_fromhtml('/home/kassa/Downloads/StrataScratch_files/orders.html')

orders['order_date'], orders['cust_id'] = orders['order_date'].apply(pd.to_datetime), orders['cust_id'].astype(int)

orders['month'] =  orders['order_date'].dt.month 
orders = orders[orders['month'] == 3]
df = orders.groupby('cust_id')['total_order_cost'].sum().reset_index().sort_values('total_order_cost', ascending = False)
df.head()

,cust_id,total_order_cost
1,7,80
3,15,208050
2,12,20
0,3,1008030


---------------------------------
# Host Popularity Rental Prices
- Hard

You’re given a table of rental property searches by users. The table consists of search results and outputs host information for searchers. Find the minimum, average, maximum rental prices for each host’s popularity rating. The host’s popularity rating is defined as below:

    0 reviews: New
    1 to 5 reviews: Rising
    6 to 15 reviews: Trending Up
    16 to 40 reviews: Popular
    more than 40 reviews: Hot

Tip: The `id` column in the table refers to the search ID. You'll need to create your own host_id by concating price, room_type, host_since, zipcode, and number_of_reviews.

Output host popularity rating and their minimum, average and maximum rental prices.

# Approaches
- Because this is a table of searches by users looking for a rental property, you can’t assume that hosts are listed distinctly, meaning that there could be the same hosts that appear in multiple searches. So taking the average, min, max without de-duplicating would skew the results. You’ll need to create a few where each host is listed only once.

- Using the new view of distinct hosts, categorize the hosts by their popularity rating defined by their number of reviews.

- Find the average, min, max price by popularity rating


In [50]:
import pandas as pd
df = getdata_fromhtml('/home/kassa/Downloads/StrataScratch_files/airbnb_host_searches.html')
df = df.drop_duplicates(subset = ['price', 'zipcode', 'host_since', 'number_of_reviews', 'room_type'])

df['number_of_reviews'], df['price'] = df['number_of_reviews'].astype(int), df['price'].astype(float)

def to_rating(n):
    if n == 0: return 'New'
    elif n <= 5: return 'Rising'
    elif n <= 15: return 'Trending up'
    elif n <= 40: return 'popular'
    elif n > 40: return 'Hot'
    else:
        return 'Error'

df['rating'] = df['number_of_reviews'].apply(to_rating)
df.groupby('rating').price.agg(['min', 'mean', 'max']).reset_index().head()




,rating,min,mean,max
0,Hot,340.12,464.233158,633.51
1,New,313.55,515.919714,741.76
2,Rising,355.53,503.846585,717.01
3,Trending up,361.09,476.277179,685.65
4,popular,270.81,472.815000,667.83


-------------------------------------
# Popularity of Hack
- Easy

Meta/Facebook has developed a new programing language called Hack.To measure the popularity of Hack they ran a survey with their employees. The survey included data on previous programing familiarity as well as the number of years of experience, age, gender and most importantly satisfaction with Hack. Due to an error location data was not collected, but your supervisor demands a report showing average popularity of Hack by office location. Luckily the user IDs of employees completing the surveys were stored.

Based on the above, find the average popularity of the Hack per office location. Output the location along with the average popularity.

# Approaches
- Perform an inner merge on dataframe using pd.merge(df1,df2, left_on = common_key_left_df, right_on = common_key_right_table, how = 'inner')

- Use .groupby(column_name) to group the dataframe about the specifed column and use mean() to get the average value per group

In [54]:
facebook_employees = getdata_fromhtml('/home/kassa/Downloads/StrataScratch_files/facebook_employees.html')
facebook_hack_survey = getdata_fromhtml('/home/kassa/Downloads/StrataScratch_files/facebook_hack_survey.html')

df = pd.merge(facebook_employees, facebook_hack_survey, left_on = 'id', right_on = 'employee_id', how = 'inner')
df[['id', 'employee_id']] = df[['id', 'employee_id']].astype(int)
df['popularity'] = df['popularity'].astype(float)


merged_df1 = df.groupby('location')['popularity'].mean().reset_index()
merged_df1.head()

,location,popularity
0,India,7.500000
1,Switzerland,1.000000
2,UK,4.333333
3,USA,4.600000


----------------------------------
# Most Active Users On Messenger
- Medium

Meta/Facebook Messenger stores the number of messages between users in a table named 'fb_messages'. In this table 'user1' is the sender, 'user2' is the receiver, and 'msg_count' is the number of messages exchanged between them.

Find the top 10 most active users on Meta/Facebook Messenger by counting their total number of messages sent and received. Your solution should output usernames and the count of the total messages they sent or received

# Approach
- Concatenate two slices of the dataframe

- Aggregated msg_counts by all the users using group by function

- Sort users by their total number of messages in descending order


In [58]:
import pandas as pd
import datetime as dt


df = getdata_fromhtml('/home/kassa/Downloads/StrataScratch_files/fb_messages.html')
df['date'], df['msg_count'] = df['date'].apply(pd.to_datetime), df['msg_count'].astype(int)


df_user1 = df[['date', 'user1', 'msg_count']]
df_user2 = df[['date', 'user2', 'msg_count']]
df = pd.concat([df_user1, df_user2.rename(columns = {'user2':'user1'})], ignore_index = True)

df = df.groupby('user1')['msg_count'].sum().to_frame('tota_no_mesa').reset_index()
df['ranking'] = df.tota_no_mesa.rank(method = 'first', ascending = False)

resul = df[df['ranking']<= 10]
resul = resul[['user1', 'tota_no_mesa']]
resul.head()

,user1,tota_no_mesa
6,craig23,43
9,herringcarlos,37
14,jennifer11,28
17,johnmccann,47
23,lfisher,29


--------------------------
# Average Salaries
- Easy

Compare each employee's salary with the average salary of the corresponding department.
Output the department, first name, and salary of employees along with the average salary of that department.

# Approach
- Create a new column that contains the average salary per group by using mean() within the transform() function to recombine the mean values to the grouped values
- Use [ [ column_name/s] ] to return a specified column of the dataframe

In [62]:
import pandas as pd

df = getdata_fromhtml('/home/kassa/Downloads/StrataScratch_files/employee.html')
df['salary'] = df['salary'].astype(float)
df['avg_salary'] = df.groupby(['department'])['salary'].transform('mean')

new_df = df[['department', 'first_name', 'salary', 'avg_salary']]

new_df.head()

,department,first_name,salary,avg_salary
0,Sales,Max,1300.0,1336.363636
1,Management,Katty,150000.0,175000.000000
2,Management,Richerd,250000.0,175000.000000
3,Sales,Jennifer,1000.0,1336.363636
4,Management,George,100000.0,175000.000000


----------------
# Spam Posts
-Medium 

Calculate the percentage of spam posts in all viewed posts by day. A post is considered a spam if a string "spam" is inside keywords of the post. Note that the facebook_posts table stores all posts posted by users. The facebook_post_views table is an action table denoting if a user has viewed a post.

# Approach
- Create a separate column in order to mark spam posts

- Use inner merges to find spam posts and the number of total viewed posts in order to calculate the spam viewed ratio per day.

- Group by with two operations at time: sum of spam (assuming spam is 1 and non-spam is 0) and count using agg method with dictionary as argument

In [64]:
import pandas as pd

# Start writing code
facebook_posts = getdata_fromhtml('/home/kassa/Downloads/StrataScratch_files/facebook_posts.html')
facebook_post_views = getdata_fromhtml('/home/kassa/Downloads/StrataScratch_files/facebook_post_views.html')
merged = pd.merge(facebook_posts, facebook_post_views, on = 'post_id', how = 'inner')
#merged['date'] = merged['post_date'].dt.to_period('D')
merged_spam = merged[merged['post_keywords'].str.contains('spam')]
merged['post_date'] = merged['post_date'].apply(pd.to_datetime)

df= merged.groupby('post_date')['post_keywords'].count().to_frame('tot_cnt').reset_index()
df_spam = merged_spam.groupby('post_date')['post_keywords'].count().to_frame('spa_cnt').reset_index()

df['percentage'] = 100*df_spam['spa_cnt']/df['tot_cnt']
resu = df[['post_date', 'percentage']]

resu.head()

,post_date,percentage
0,2019-01-01,100.0
1,2019-01-02,50.0


------------------
# Total Cost Of Orders
- Easy

Find the total cost of each customer's orders. Output customer's id, first name, and the total order cost. Order records by customer's first name alphabetically.

# Approach
- Use the SUM() function to find the total cost.

- Use JOIN on customer id to combine records from both tables.

- Group records by the customer's first name.

In [66]:

import pandas as pd

customers = getdata_fromhtml('/home/kassa/Downloads/StrataScratch_files/customers.html')
orders = getdata_fromhtml('/home/kassa/Downloads/StrataScratch_files/orders.html')

merged = pd.merge(customers, orders, left_on = 'id', right_on = 'cust_id', how = 'inner')

merged = merged.groupby(['cust_id', 'first_name'])['total_order_cost'].sum().reset_index()
merged = merged.sort_values('first_name')
merged.head()

,cust_id,first_name,total_order_cost
0,12,Eva,6020125
2,3,Farida,100803050
4,5,Henry,80
5,7,Jill,258015050305025125
1,15,Mia,1006020805030200
3,4,William,8060


------------------
# Classify Business Type
- Medium

Classify each business as either a restaurant, cafe, school, or other. A restaurant should have the word 'restaurant' in the business name. For cafes, either 'cafe', 'café', or 'coffee' can be in the business name. 'School' should be in the business name for schools. All other businesses should be classified as 'other'. Output the business name and the calculated classification.

# Approach

- Create new column business_type and assign categories based on substrings

- Use lambda function with criteria on substrings (‘substring’ in string)

In [69]:
import pandas as pd
df = getdata_fromhtml('/home/kassa/Downloads/StrataScratch_files/sf_restaurant_health_violations.html')

def to_clasify(s):
    if 'school' in s.lower(): return 'school'
    elif 'restaurant' in s.lower(): return 'restaurant'
    elif ('cafe' in s.lower()) or ('café' in s.lower()) or ('coffee' in s.lower()): return 'cafe'
    else: return 'other'
df['category'] = df['business_name'].apply(to_clasify)
df[['business_name', 'category']].drop_duplicates().head()

,business_name,category
0,John Chin Elementary School,school
1,Sutter Pub and Restaurant,restaurant
2,SRI THAI CUISINE,other
3,Washington Bakery & Restaurant,restaurant
4,Brothers Restaurant,restaurant


----------------------
# Comments Distribution
- Hard 

Write a query to calculate the distribution of comments by the count of users that joined Meta/Facebook between 2018 and 2020, for the month of January 2020. 

The output should contain a count of comments and the corresponding number of users that made that number of comments in Jan-2020. For example, you'll be counting how many users made 1 comment, 2 comments, 3 comments, 4 comments, etc in Jan-2020. Your left column in the output will be the number of comments while your right column in the output will be the number of users. Sort the output from the least number of comments to highest.

To add some complexity, there might be a bug where an user post is dated before the user join date. You'll want to remove these posts from the result.

# Approach

- Join users and comments table using an inner merge

- Filter the dataset for comments from 01-2020 

- Filter the dataset for users that registered between 2018 and 2020.

- Exclude potential bugs from the dataset where comments that are created before user joined should be excluded

- Count number of comments per user for Jan-2020

- The output should be one column with the count of comments and another column with a count of users that made a certain number of comments in Jan-2020

In [73]:
path1 = '/home/kassa/Downloads/StrataScratch_files/fb_users.html'
path2 = '/home/kassa/Downloads/StrataScratch_files/fb_comments.html'

fb_users = getdata_fromhtml(path1)
fb_comments = getdata_fromhtml(path2)

merged = pd.merge(fb_users, fb_comments, left_on = 'id', right_on = 'user_id', how = 'inner')

#
df = merged
df_jan = df[df['created_at'].between('2020-01-01', '2020-01-31')]
df_jan = df_jan[df_jan['joined_at'].between('2018-01-01', '2020-12-31')]

df_jan = df_jan[df_jan['created_at'] > df_jan['joined_at']]

#df_jan['cnt'] = 1
df_jan = df_jan.groupby('user_id')['body'].count().to_frame('cnt_comments').reset_index()

df_fin = df_jan.groupby('cnt_comments')['user_id'].count().reset_index()
resul = df_fin.sort_values('cnt_comments')

resul.head()

,cnt_comments,user_id
0,1,4
1,2,6
2,3,1
3,4,1
4,6,1


---------------
# Top Cool Votes
- Medium

Find the review_text that received the highest number of  'cool' votes. Output the business name along with the review text with the highest numbef of 'cool' votes.

# Approach
- Select specific column/s from dataframe using [column_name] then select rows with values equal to ‘==’ maximum value of cool

- use max() to return the maximum or highest value

- Use [ [ column_name/s] ] to return a specified column of the dataframe


In [77]:
path = '/home/kassa/Downloads/StrataScratch_files/yelp_reviews.html'
yelp_reviews = getdata_fromhtml(path)
yelp_reviews['cool'] = yelp_reviews['cool'].astype(float)

cool_max = yelp_reviews[yelp_reviews['cool'] == yelp_reviews['cool'].max()]

cool_max = cool_max[['business_name', 'review_text']]
cool_max.head()

,business_name,review_text
30,Roka Akor,"I\n hate to admit it, but it had been a long w..."
105,Lunardis,This is the nicest grocery store in the city. ...


------------------
# Order Details
- Medium

Find order details made by Jill and Eva. Consider the Jill and Eva as first names of customers. Output the order date, details and cost along with the first name. Order records based on the customer id in ascending order.

# Approach
- Perform inner join on orders and customers using pd.merge(dataframe1, dataframe2, on = common_table_keys)

- Define or create a list containing the customer first_name to select

- Use .isin(customer_name) to filter rows if it is in the defined list

- Use [ [ column_name/s] ] to return a specified column of the dataframe

In [79]:
import pandas as pd

customers = getdata_fromhtml('/home/kassa/Downloads/StrataScratch_files/customers.html')
orders = getdata_fromhtml('/home/kassa/Downloads/StrataScratch_files/orders.html')

merged = pd.merge(customers, orders, left_on = 'id', right_on = 'cust_id', how = 'inner')

#mergerd = merged[(merged['first_name'] == 'Jill') or (merged['first_name'] == 'Eva')]

merged = merged[merged['first_name'].isin(['Jill', 'Eva'])].sort_values('cust_id')
merged = merged[['order_date', 'order_details', 'total_order_cost', 'first_name']]

merged.head()

,order_date,order_details,total_order_cost,first_name
18,2019-01-11,Shirts,60,Eva
19,2019-03-11,Slipper,20,Eva
20,2019-01-11,Coat,125,Eva
0,2019-02-01,Coat,25,Jill
1,2019-03-10,Shoes,80,Jill


-----------------------
# Revenue Over Time
- Hard

Find the 3-month rolling average of total revenue from purchases given a table with users, their purchase amount, and date purchased. Do not include returns which are represented by negative purchase values. Output the year-month (YYYY-MM) and 3-month rolling average of revenue, sorted from earliest month to latest month.

A 3-month rolling average is defined by calculating the average total revenue from all user purchases for the current month and previous two months. The first two months will not be a true 3-month rolling average since we are not given data from last year. Assume each month has at least one purchase.

# Approach
- Subset the data frame for only positive purchase_amt

- Create a derived variable year_month (YYYY-MM)

- Aggregate purchase amount as monthly total using groupby() function, output should be month and total purchase amount and sort values by month in ascending order using sort_values()

- Use a window frame of 3 months -- the current month and the previous 2 months using rolling function, min_periods 1 means 1st and second row will have average of 1 an d2 values respectively. Row 3 onwards, values are average of 3 values.

- Use a mean() function to calculate average purchase amount over the defined window frame

In [83]:
import pandas as pd
import datetime as dt

# Start writing code
df = getdata_fromhtml('/home/kassa/Downloads/StrataScratch_files/amazon_purchase.html')
df['purchase_amt'] = df['purchase_amt'].astype(float)
df['created_at'] = df['created_at'].apply(pd.to_datetime)

df = df[df['purchase_amt'] > 0]
df['ym'] = df['created_at'].dt.to_period('M')
df
df = df.groupby(['ym'])['purchase_amt'].sum().reset_index()
df = df.sort_values('ym')
df['rolling_avg'] = df['purchase_amt'].rolling(window = 3, min_periods = 1).mean()
result = df[['ym', 'rolling_avg']]

result.head()

,ym,rolling_avg
0,2020-01,26292.000000
1,2020-02,23493.500000
2,2020-03,25535.666667
3,2020-04,24082.666667
4,2020-05,25417.666667


------------------
# Workers With The Highest Salaries
- Medium 

Find the titles of workers that earn the highest salary. Output the highest-paid title or multiple titles that share the highest salary.

# Approach

- In the subquery prepare the list of work titles with the highest salary.

In [4]:
import pandas as pd
from getdata_fromhtml import getdata_fromhtml as g
# Start writing code
worker = g('/home/kassa/Downloads/StrataScratch_files/worker.html')
title = g('/home/kassa/Downloads/StrataScratch_files/title.html')

merged = pd.merge(worker, title, left_on = 'worker_id', right_on = 'worker_ref_id', how = 'inner')
merged[['worker_id', 'worker_ref_id']] = merged[['worker_id', 'worker_ref_id']].astype(int)
merged['salary'] = merged['salary'].astype('float')

merged = merged[merged['salary'] == merged.salary.max()]['worker_title']
merged.head()

3    Asst. Manager
4          Manager
Name: worker_title, dtype: object

# Reviews of Categories
- Medium

Find the top business categories based on the total number of reviews. Output the category along with the total number of reviews. Order by total reviews in descending order.

# Approach
- Use split and explode in order to convert multiple categories field into single category field.

In [7]:
from getdata_fromhtml import getdata_fromhtml as g
path = '/home/kassa/Downloads/StrataScratch_files/yelp_business.html'

df = g(path)
#

df = df[['review_count', 'categories']].set_index('review_count').apply(lambda x: x.str.split(';').explode()).reset_index()
df = df.groupby('categories')['review_count'].sum().reset_index().sort_values('review_count', ascending = False)

df.head()

,categories,review_count
23,Buffets,87
44,Desserts,87
167,Traditional Chinese Medicine,86
55,Eyewear & Opticians,852
121,Optometrists,852


# Distances Traveled
- Medium

Find the top 10 users that have traveled the greatest distance. Output their id, name and a total distance traveled.

# Appoach

- Merge lyft_users and lyft_rides_log tables to get all the information using an inner merge

- Group by user_id to count total distance traveled by each user

- Sum distances per user to get total traveled distance for each user

- Sort the Dataframe by distance in descending order and select only top 10 rows by distance travelled

- Return a dataframe with 2 columns (user_id, distance)


In [9]:
import pandas as pd
from getdata_fromhtml import getdata_fromhtml as g
lyft_users = g('/home/kassa/Downloads/StrataScratch_files/lyft_users.html')
lyft_rides_log = g('/home/kassa/Downloads/StrataScratch_files/lyft_rides_log.html')

merged = pd.merge(lyft_users, lyft_rides_log, left_on ='id', right_on = 'user_id', how = 'inner')
df = merged.groupby(['user_id', 'name'])['distance'].sum().reset_index()
df = df.sort_values('distance', ascending = False)
df.iloc[0:10, :].head()

,user_id,name,distance
16,26,Barbara Larson,9894
40,7,Christopher Schmitt,96
36,48,Pamela Cox,942958
41,8,Patrick Gutierrez,93477429
38,5,Sean Parker,92


--------------------------------------
# Highest Salary In Department
- Medium (twitter)

Find the employee with the highest salary per department. Output the department name, employee's first name along with the corresponding salary.

# Approach
- Using the Indexing method that is df[df[column]== condition] to filter out specific row values, specify the condition being the highest salary per department by using groupby() then applying transform(max) to look for this values per group in the dataframe

- Use [ [ column_name/s] ] to return a specified column of the dataframe


In [13]:
from getdata_fromhtml import getdata_fromhtml as g
df = g('/home/kassa/Downloads/StrataScratch_files/employee.html')
df = df[df.groupby(['department'])['salary'].transform('max')==df['salary']]
resu = df[['first_name', 'department', 'salary']]  #transform('max')
resu.head()

,first_name,department,salary
2,Richerd,Management,250000
18,Mick,Sales,2200
22,Michale,Audit,700


----------------------------
# Gender With Generous Reviews
- Easy (airbnb)

Write a query to find which gender gives a higher average review score when writing reviews as guests. Use the `from_type` column to identify guest reviews. Output the gender and their average review score.

# Approach

- Perform inner join using pd.merge(dataframe1, dataframe2, on = common key)

- Filter specific column/s from dataframe using [column_name] then select rows with values equal to ‘==’ guest

- Use .groupby(column_name) on gender to group the dataframe about the specifed column and use mean() to get the average per group; Convert the resulting object to a dataframe using to_frame('column_name')

- Select the gender with the highest average score using max() and ['column_name'] to return only the gender column

In [19]:
from getdata_fromhtml import getdata_fromhtml as g
airbnb_reviews = g('/home/kassa/Downloads/StrataScratch_files/airbnb_reviews.html')
airbnb_guests = g('/home/kassa/Downloads/StrataScratch_files/airbnb_guests.html')
df = pd.merge(airbnb_reviews,airbnb_guests, left_on = 'from_user', right_on = 'guest_id', how = 'inner')

df['review_score'] = df['review_score'].astype(float)
df = df[df['from_type']=='guest']
df = df.groupby('gender')['review_score'].mean().reset_index()
df = df[df['review_score'] == df['review_score'].max()]

df.head()

,gender,review_score
1,M,5.526316


--------------------------------------
# Expensive Projects
- Easy (Microsoft)

Given a list of projects and employees mapped to each project, calculate by the amount of project budget allocated to each employee. The output should include the project title and the project budget per employee rounded to the closest integer. Order your list by projects with the highest budget per employee first.

# Approach
- To calculate the amount of budget allocated to each employee (budget-to-employee ratio), you'll need to determine the project budget and divide by the number of employees on the project

- Remove projects with no employees by linking the two tables and filtering using an INNER JOIN



In [22]:
from getdata_fromhtml import getdata_fromhtml as g

ms_projects = g('/home/kassa/Downloads/StrataScratch_files/ms_projects.html')
ms_emp_projects = g('/home/kassa/Downloads/StrataScratch_files/ms_emp_projects.html')
merged = pd.merge(ms_projects, ms_emp_projects, left_on = 'id', right_on = 'project_id', how = 'inner')

merged[['emp_id', 'project_id']] = merged[['emp_id', 'project_id']].astype(int)
merged['budget'] = merged['budget'].astype(float)

merged['n_emp'] = merged.groupby('title')['emp_id'].transform('count')#.reset_index()
merged['budget_to_emp_ratio'] = merged['budget']/merged['n_emp']
resu = merged[['title', 'budget_to_emp_ratio']]
resu = resu.sort_values('budget_to_emp_ratio', ascending = False).drop_duplicates()
resu = round(resu)
resu.head()

,title,budget_to_emp_ratio
15,Project8,24642.0
97,Project49,24387.0
28,Project15,24058.0
18,Project10,23794.0
37,Project19,22493.0


--------------------------
# 3 Bed Minimum
- Medium 

Find the average number of beds in each neighborhood that has at least 3 beds in total. Output results along with the neighborhood name and sort the results based on the number of average beds in descending order.

# Approach

- Use .groupby(column_name) to group the dataframe about the specifed column then  filter out values per group by using filter() with then specify a lambda function where in (lambda x: x[column] == condition) 

-  Use groupby again then apply [column1].mean() function to get the mean of values of the column

- Use [ [ column_name/s] ] to return a specified column of the dataframe

- Use sort_values(column_name, order) to sort along a specified column; Set order to False to display the printed values in descending order


In [25]:
from getdata_fromhtml import getdata_fromhtml as g
df = g('/home/kassa/Downloads/StrataScratch_files/airbnb_search_details.html')
df['beds'] = df['beds'].astype(int)

df = df.groupby('neighbourhood').filter(lambda x: x['beds'].sum() >= 3)
df = df.groupby(['neighbourhood'])['beds'].mean().to_frame('avg_no_beds').reset_index()
df = df.sort_values('avg_no_beds', ascending= False)
df.head()


,neighbourhood,avg_no_beds
18,Pacific Palisades,6.0
28,Windsor Terrace,5.0
10,Hayes Valley,5.0
20,Redondo Beach,5.0
23,The Rockaways,4.0


----------------------
# Cum Sum Energy Consumption
- Hard

Calculate the running total (i.e., cumulative sum) energy consumption of the Meta/Facebook data centers in all 3 continents by the date. Output the date, running total energy consumption, and running total percentage rounded to the nearest whole number.

# Approach
- Think about which method fits best in this exercise: append, join, merge, concat

- Use sum of consumption as an additional column

- Use cumsum() to get the cumulative sum

- Use [ [column_name] ] to return a specified column of the dataframe

- Use to_datetime() to convert timestamp to date format

In [29]:
from getdata_fromhtml import getdata_fromhtml as g
import datetime as dt

fb_eu_energy = g('/home/kassa/Downloads/StrataScratch_files/fb_eu_energy.html')
fb_na_energy = g('/home/kassa/Downloads/StrataScratch_files/fb_na_energy.html')
fb_asia_energy = g('/home/kassa/Downloads/StrataScratch_files/fb_asia_energy.html')

merged = pd.concat([fb_eu_energy, fb_na_energy, fb_asia_energy])
merged['date'], merged['consumption'] = merged['date'].apply(pd.to_datetime), merged['consumption'].astype(float)

merged  = merged.groupby('date')['consumption'].sum().to_frame('total_consumption').reset_index()
merged['cum_sum'] = merged['total_consumption'].cumsum()
 
tota = merged['total_consumption'].sum()

merged['percentage'] =  round(100*merged['cum_sum']/tota)#.round()
merged['date'] = pd.to_datetime(merged['date'], format = '%Y-%m-%d').dt.strftime('%Y-%m-%d')

result = merged[['date', 'cum_sum', 'percentage']]
result.head()

,date,cum_sum,percentage
0,2020-01-01,1050.0,13.0
1,2020-01-02,2175.0,27.0
2,2020-01-03,3275.0,40.0
3,2020-01-04,4450.0,55.0
4,2020-01-05,5650.0,69.0


-------------------------
# Employee and Manager Salaries
- Medium (dropbox)

Find employees who are earning more than their managers. Output the employee's first name along with the corresponding salary. Find employees who are earning more than their managers. Output the employee's first name along with the corresponding salary.

# Approach
- Perform self join to itself using pd.merge(dataframe1, dataframe2, on = common_table_keys); By default would add '_x' and '_y' suffixes to duplicate columns

- Filter out rows where salary_x is greater than salary_y

- Use [ [ column_name/s] ] to return a specified column of the dataframe

In [32]:
from getdata_fromhtml import getdata_fromhtml as g
df = g('/home/kassa/Downloads/StrataScratch_files/employee.html')
df['salary'] = df['salary'].astype(float)

merged = pd.merge(df, df, right_on = 'id', left_on='manager_id')
merged = merged[merged['salary_x'] > merged['salary_y']][['first_name_x', 'salary_x']]
merged.head()

,first_name_x,salary_x
2,Richerd,250000.0


---------------
# Percentage Of Total Spend
- Medium (Amazon)

Calculate the percentage of the total spend a customer spent on each order. Output the customer’s first name, order details, and percentage of the order cost to their total spend across all orders. 

Assume each customer has a unique first name (i.e., there is only 1 customer named Karen in the dataset) and that customers place at most only 1 order a day.

Percentages should be represented as fractions

# Approach
-  First join the `customer` table with the `orders` table using an inner merge. Any customers that did not place an order will be removed in the resulting dataset.

- You’ll want to sum the total order cost by customer and their order details. This is essentially your numerator.

- Next, you'll want to sum the total spend across all orders for each customer. This will be your denominator.

- To find the percentage total cost for each order, divide the numerator by denominator. Multiple by 100 to convert to a percentage and round the number to a whole number.


In [38]:
from getdata_fromhtml import getdata_fromhtml as g
orders = g('/home/kassa/Downloads/StrataScratch_files/orders.html')
customers = g('/home/kassa/Downloads/StrataScratch_files/customers.html')

df = pd.merge(orders, customers, left_on ='cust_id', right_on = 'id', how = 'inner')
df[['cust_id', 'id_y']], df['total_order_cost'] = df[['cust_id', 'id_y']].astype(int), df['total_order_cost'].astype(float)
df['percentage'] = df['total_order_cost']/df.groupby(['cust_id'])['total_order_cost'].transform('sum')

df = df[['first_name', 'order_details', 'percentage']]
df.head()

,first_name,order_details,percentage
0,Farida,Coat,0.384615
1,Farida,Shoes,0.307692
2,Farida,Skirt,0.115385
3,Farida,Shirts,0.192308
4,Jill,Coat,0.046729


-----------------
# New Products

You are given a table of product launches by company by year. Write a query to count the net difference between the number of products companies launched in 2020 for the first time with the number of products companies launched in the previous year. Output the name of the companies and a net difference of net products released for 2020 compared to the previous year. If a company is new or had no products in 2019, then any product released in 2020 would be considered as new.

# Approach 

- Create 2 data frames where you're filtering the table for launches by 2020 and 2019

- Calculate distinct model launches in 2020 with 2029 using groupby and nunique and save the output in new data frames respectively

- OUTER JOIN 2020 grouped data frame with 2019 data frame

- Derive a new variable with difference in distinct launch count in 2 consecutive years

- Filter out all the company name where this difference greater than 0

- Output should be company name sorted in ascending order

In [42]:
from getdata_fromhtml import getdata_fromhtml as g
df = g('/home/kassa/Downloads/StrataScratch_files/car_launches.html')
df['year'] = df['year'].apply(pd.to_numeric)

df_19 = df[df['year'] == 2019].add_prefix('2019_')
df_19 = df_19.groupby('2019_company_name')['2019_product_name'].nunique().to_frame('nunique_prod_19').reset_index()

df_20 = df[df['year'] == 2020].add_prefix('2020_')
df_20 = df_20.groupby('2020_company_name')['2020_product_name'].nunique().to_frame('nunique_prod_20')

merged = pd.merge(df_19, df_20, left_on = '2019_company_name', right_on = '2020_company_name', how= 'outer')
merged['diff'] = merged['nunique_prod_20'] - merged['nunique_prod_19'] 
#merged = merged[merged['diff'] >= 0]
res = merged[['2019_company_name', 'diff']]
res.sort_values('2019_company_name')

,2019_company_name,diff
0,Chevrolet,2
1,Ford,-1
2,Honda,-3
3,Jeep,1
4,Toyota,-1


-------------------
# Time Between Two Events
- Hard 

Meta/Facebook's web logs capture every action from users starting from page loading to page scrolling. Find the user with the least amount of time between a page load and their first scroll down. Your output should include the user id, page load time, first scroll down time, and time between the two events in seconds.

# Approach

- Collect load time and scroll time as separate fields

- Calculate the time difference between them

- Select the lowest variance

- This approach is not sensitive to situation when there are multiple operation of the same kind in row

In [45]:
from getdata_fromhtml import getdata_fromhtml as g
import datetime as dt

df = g('/home/kassa/Downloads/StrataScratch_files/facebook_web_logs.html') 
df['timestamp'] = df['timestamp'].apply(pd.to_datetime)
page_load = df[df['action'] == 'page_load'] 
page_scrol = df[df['action'] == 'scroll_down'] 
merged = pd.merge(page_load, page_scrol, on = 'user_id', how = 'inner')

merged['time_diff'] = (merged['timestamp_y'] - merged['timestamp_x']).dt.seconds

resu = merged[merged['time_diff'] == merged['time_diff'].min()]
resu = resu[['user_id', 'timestamp_x', 'timestamp_y', 'time_diff']]
#resu.drop_duplicates()
resu.head()

,user_id,timestamp_x,timestamp_y,time_diff
30,2,2019-04-25 13:41:21,2019-04-25 13:41:30,9


--------------------
# Number of violations
- Medium (City of San Fransisco)

You're given a dataset of health inspections. Count the number of violation in an inspection in 'Roxanne Cafe' for each year. If an inspection resulted in a violation, there will be a value in the 'violation_id' column. Output the number of violations by year in ascending order.

# Approach
- Extract the year from the inspection date.

- Count the number of records after filtering for Roxanne Cafe and where there is a violation for the inspection

In [48]:
from getdata_fromhtml import getdata_fromhtml as g
import datetime as dt
df = g('/home/kassa/Downloads/StrataScratch_files/sf_restaurant_health_violations.html')
df['inspection_date'] = df['inspection_date'].apply(pd.to_datetime)
df['year'] =  df['inspection_date'].dt.year
df = df[df['business_name'] == 'Roxanne Cafe']
df = df.groupby('year')['inspection_id'].count().reset_index().sort_values('year')
df.head()

,year,inspection_id
0,2015,5
1,2016,2
2,2018,3


-------------------
# Growth of Airbnb
- Hard

Estimate the growth of Airbnb each year using the number of hosts registered as the growth metric. The rate of growth is calculated by taking ((number of hosts registered in the current year - number of hosts registered in the previous year) / the number of hosts registered in the previous year) * 100.

Output the year, number of hosts in the current year, number of hosts in the previous year, and the rate of growth. Round the rate of growth to the nearest percent and order the result in the ascending order based on the year.

Assume that the dataset consists only of unique hosts, meaning there are no duplicate hosts listed.

# Approach
- Filter out rows that are notnull() or "!=" '' empty string

 - Use .groupby(column_name) to group the dataframe about the specifed column and use size() to get the number of elements in the specified column

 - Use shift(n) to offset the row values by n count

 - Use rount(int, decimal_places) to round the result to 2 digits

 - convert it to string using astype(str) to concatenate '%' by using operator '+' airbnb_search_details 


In [52]:
import datetime as dt
from getdata_fromhtml import getdata_fromhtml as g
df = g('/home/kassa/Downloads/StrataScratch_files/airbnb_search_details.html')
df['host_since'] = df['host_since'].apply(pd.to_datetime)

df = df[df.notnull()]
df['year'] = df['host_since'].dt.year
df_new = df.groupby('year').size().to_frame('nhost').reset_index()
df_new['prev_cnt'] = df_new['nhost'].shift(1)
df_new['growth_rate'] = round(100*(df_new['nhost']-df_new['prev_cnt'])/df_new['prev_cnt'])
df_new.head()

,year,nhost,prev_cnt,growth_rate
0,2009,2,NaN,NaN
1,2010,4,2.0,100.0
2,2011,9,4.0,125.0
3,2012,10,9.0,11.0
4,2013,30,10.0,200.0


-------------
# Customer Orders and Details
- Medium (Amazon)

Find the number of orders, the number of customers, and the total cost of orders for each city. Only include cities that have made at least 5 orders and count all customers in each city even if they did not place an order.

Output each calculation along with the corresponding city name.

# Approach
- Use pd.merge(dataframe1, dataframe2, on = common_table_keys, how = 'left') to perform left join on the dataframes

- Use groupby() to group according to a specific coulumn then apply agg() to perform or use multiple operations on several columns simultaneously - count to get the count of values of cust_id and id_x, sum of order_cost


In [57]:
from getdata_fromhtml import getdata_fromhtml as g
customers = g('/home/kassa/Downloads/StrataScratch_files/customers.html')
orders = g('/home/kassa/Downloads/StrataScratch_files/orders.html')

customers['id'], orders['cust_id'] = customers['id'].astype(int), orders['cust_id'].astype(int)
merged = pd.merge(customers, orders, left_on ='id', right_on = 'cust_id', how= 'left')
merged['total_order_cost'] = merged['total_order_cost'].astype(float)
 
df = merged.groupby('city').agg({'id_x': lambda x: x.nunique(), 'cust_id': 'count',  'total_order_cost': 'sum'}).reset_index()

df = df[df['cust_id'] >= 5]
#df = df.rename({'cust_id': 'id_c', 'order_date': 'id_o'}, axis= 1)


resu = df[['city', 'id_x', 'cust_id', 'total_order_cost']]
resu.head()

,city,id_x,cust_id,total_order_cost
1,Austin,1,8,535.0
4,Miami,5,8,620.0


----------------------
# Day 1 Common Reactions
- Medium

Find the most common reaction for day 1 by counting the number of occurrences for each reaction. Output the reaction alongside its number of occurrences.

# Approach 
- Use .groupby(column_name) to group the dataframe about the specifed column

- Use size() to get the number of elements per group then convert the resulting object to a dataframe using to_frame()

- Use sort_values(column_name, order) to sort along a specified column; Set order to False to display the printed values in descending order 

- Filter the dataframe by maximum value of the occurences

In [63]:
from getdata_fromhtml import getdata_fromhtml as g

df = g('/home/kassa/Downloads/StrataScratch_files/facebook_reactions.html')
df['date_day'] = df['date_day'].apply(pd.to_numeric)
df_d1 = df[df['date_day'] == 1]
res = df_d1.groupby('reaction').size().to_frame('n_reaction').reset_index()
res = res[res['n_reaction'] == res['n_reaction'].max()]
res.head()

,reaction,n_reaction
1,laugh,5
2,like,5


----------------------
# Unique Salaries
- Hard (Twitter)

Find the top three unique salaries for each department. Output the department name and the top 3 unique salaries by each department. Order your results alphabetically by department and then by highest salary to lowest.

# Approach
- Remove duplicates on department and salary by using df.drop_duplicates()

- Use groupby() to group per department and nlargest(3) to return the top 3 highest salary per group

- Convert the resulting object to a dataframe using to_frame()

- Perform left join on the resulting highest salary per group dataframe and the distinct department and salary dataframe by using pd.merge(dataframe1, dataframe2, on = ['salary','department'], how = 'left') to determine the other attributes of the highest salary per department

- Use [ [ column_name/s] ] to return a specified column of the dataframe


In [66]:
from getdata_fromhtml import getdata_fromhtml as g
df = g('/home/kassa/Downloads/StrataScratch_files/twitter_employees.html')
df = df.drop_duplicates() 
df['salary'] = df['salary'].apply(pd.to_numeric)
def get_slice(df):
    x_vals = df['salary'].drop_duplicates().nlargest(3)
    return  x_vals

df = df.groupby('department').apply(get_slice).reset_index()
df.head()

,department,level_1,salary
0,Audit,24,110000
1,Audit,25,100000
2,Audit,27,70000
3,Management,3,250000
4,Management,0,200000


--------------
# Cities With The Most Expensive Homes
- Medium (Zillow)

Write a query that identifies cities with higher than average home prices when compared to the national average. Output the city names.

# Approach
- Calculate average price by city using groupby
- Calculate average price of whole data frame
- Create a subset of data frame where city average is higher than overall average

In [68]:
from getdata_fromhtml import getdata_fromhtml as g

df = g('/home/kassa/Downloads/StrataScratch_files/zillow_transactions.html')
df['mkt_price'] = df['mkt_price'].apply(pd.to_numeric)
avg_price = sum(df['mkt_price'])/df.shape[0]

df = df.groupby('city')['mkt_price'].mean().to_frame('mean_price').reset_index()
df = df[df['mean_price'] > avg_price]
resu = df['city']
resu.head()

3    Mountain View
8    San Francisco
9      Santa Clara
Name: city, dtype: object

---------------------
# User Email Labels
- Medium (Google)

Find the number of emails received by each user under each built-in email label. The email labels are: 'Promotion', 'Social', and 'Shopping'. Output the user along with the number of promotion, social, and shopping mails count.

# Approach
- Perform inner join on google_emails and google_labels df using pd.merge(df1, df2, left_on = common_key_left_df, right_on = common_key_right_table)

- Use .groupby(column_name) to group the dataframe about the specifed column and use size() to count of values per group. Do this for both dataframes

- Pivot the dataframe using pd.pivot_table with to_user as the index, label as column, and the count as values; specify the aggfunc to 'sum'

- Clean multilevel index caused by the pivot by setting the columns as str using astype(str) then apply reset_index()

In [71]:
from getdata_fromhtml import getdata_fromhtml as g
google_gmail_emails = g('/home/kassa/Downloads/StrataScratch_files/google_gmail_emails.html')
google_gmail_labels = g('/home/kassa/Downloads/StrataScratch_files/google_gmail_labels.html')

df = pd.merge(google_gmail_emails, google_gmail_labels, left_on = 'id', right_on = 'email_id', how = 'inner')
df = df.groupby(['to_user', 'label']).size().to_frame('n_cnt').reset_index()
resu = pd.pivot_table(df, index = 'to_user', columns = 'label', values = 'n_cnt').reset_index()
res = resu[['to_user','Promotion', 'Shopping', 'Social']].fillna(0)
res.head()

label,to_user,Promotion,Shopping,Social
0,114bafadff2d882864,0.0,3.0,2.0
1,157e3e9278e32aba3e,1.0,1.0,2.0
2,2813e59cf6c1ff698e,2.0,2.0,4.0
3,32ded68d89443e808,3.0,0.0,1.0
4,406539987dd9b679c0,2.0,6.0,0.0


---------------------------
# Activity Rank
- Medium (google)

Find the email activity rank for each user. Email activity rank is defined by the total number of emails sent. The user with the highest number of emails sent will have a rank of 1, and so on. Output the user, total emails, and their activity rank. Order records by the total emails in descending order. Sort users with the same number of emails in alphabetical order.

In your rankings, return a unique value (i.e., a unique rank) even if multiple users have the same number of emails.

# Approach
- Use groupby() to group along from_users then use size() to get the count of values per group; convert the resulting object to dataframe using to_frame('column_name')

- Create a column that contains the rank based on the count of emails using rank(), specify parameter method='first' to get unique rank

In [73]:
from getdata_fromhtml import getdata_fromhtml as g
df = g('/home/kassa/Downloads/StrataScratch_files/google_gmail_emails.html')
df = df.groupby('from_user').size().to_frame('cnt').sort_values(['cnt', 'from_user'], ascending = [False, True]).reset_index()
df['ranks'] = df.cnt.rank(method = 'first', ascending = False)
resu = df
resu.head()

,from_user,cnt,ranks
0,32ded68d89443e808,19,1.0
1,ef5fe98c6b9f313075,19,2.0
2,5b8754928306a18b68,18,3.0
3,55e60cfcc9dc49c17e,16,4.0
4,91f59516cb9dee1e88,16,5.0


---------------------------
# Naive Forecasting
- Hard (Uber)

Some forecasting methods are extremely simple and surprisingly effective. Naïve forecast is one of them; we simply set all forecasts to be the value of the last observation. Our goal is to develop a naïve forecast for a new metric called "distance per dollar" defined as the (distance_to_travel/monetary_cost) in our dataset and measure its accuracy.



To develop this forecast,  sum "distance to travel"  and "monetary cost" values at a monthly level before calculating "distance per dollar". This value becomes your actual value for the current month. The next step is to populate the forecasted value for each month. This can be achieved simply by getting the previous month's value in a separate column. Now, we have actual and forecasted values. This is your naïve forecast. Let’s evaluate our model by calculating an error matrix called root mean squared error (RMSE). RMSE is defined as sqrt(mean(square(actual - forecast)). Report out the RMSE rounded to the 2nd decimal spot.

In [79]:
from getdata_fromhtml import getdata_fromhtml as g
df = g('/home/kassa/Downloads/StrataScratch_files/uber_requests_logs.html')

import pandas as pd
import datetime as dt
import numpy as np

df['request_date'] = df['request_date'].apply(pd.to_datetime)
df[['distance_to_travel', 'monetary_cost']] = df[['distance_to_travel', 'monetary_cost']].apply(pd.to_numeric)
df['year_month'] = df['request_date'].dt.to_period('M')
df = df.groupby('year_month').agg({'distance_to_travel': 'sum', 'monetary_cost': 'sum'}).reset_index()
df['distance_per_dollar'] = df['distance_to_travel']/df['monetary_cost']
df['prev_dist_per_dollar'] = df['distance_per_dollar'].shift()


rmse = (np.mean((df.iloc[1:, 3] - df.iloc[1:, 4])**2))**0.5 
round(rmse, 2)

2.34

-----------------
# Favorite Customer
- Medium (Amazon)

Find "favorite" customers based on the order count and the total cost of orders. A customer is considered as a favorite if he or she has placed more than 3 orders and with the total cost of orders more than $100.

Output the customer's first name, city, number of orders, and total cost of orders.

# Approach

- Perform inner join on orders and customers using pd.merge(dataframe1, dataframe2, on = common_table_keys)

- Use groupby() to group according to a specific coulumn then apply agg() to perform or use multiple operations on several columns simultaneously - count to get the count of values of id_y, sum of order_cost

In [82]:
import pandas as pd

from getdata_fromhtml import getdata_fromhtml as g

df1 = g('/home/kassa/Downloads/StrataScratch_files/customers.html')
df2 = g('/home/kassa/Downloads/StrataScratch_files/orders.html')
df1['id'] = df1['id'].astype(int)
df2['cust_id'] = df2['cust_id'].astype(int)
df2['total_order_cost'] = df2['total_order_cost'].apply(pd.to_numeric)

merged = pd.merge(df1, df2, left_on = 'id', right_on = 'cust_id', how = 'inner')
df = merged.groupby(['cust_id', 'first_name', 'city']).agg({'order_date':'count', 'total_order_cost': 'sum'}).reset_index()
df = df[(df['order_date'] > 3) & (df['total_order_cost'] >100)]
res = df[['first_name', 'city', 'order_date', 'total_order_cost']]
res.head()

,first_name,city,order_date,total_order_cost
0,Farida,San Francisco,4,260
3,Jill,Austin,8,535
5,Mia,Miami,7,540


-----------------------
# Year Over Year Churn
- Hard (lyft)

Find how the number of drivers that have churned changed in each year compared to the previous one. Output the year (specifically, you can use the year the driver left Lyft) along with the corresponding number of churns in that year, the number of churns in the previous year, and an indication on whether the number has been increased (output the value 'increase'), decreased (output the value 'decrease') or stayed the same (output the value 'no change').

# Approach
- Clean the end_date column by coverting it to datetime by applying pd.to_datetime

- Create a column that contains the count of those churned entries. Do this by making a Boolean condition where end_date is notnull() then converting it to numeric using astype(int)

- Create a column that contains the year under the end_date column using dt.year

- Group by year end using groupby() and get the sum of those who churned using sum(); convert the resulting object to a dataframe using to_frame('column_name').reset_index()

- Use shift(n) to offset alignment by n rows; replace the null values with 0

- Calculate the percentage variation as 100*(actual_number_churns - prev_year_number_churns)/prev_year_number_churns.


In [86]:
from getdata_fromhtml import getdata_fromhtml as g
import pandas as pd
import datetime as dt

df = g('/home/kassa/Downloads/StrataScratch_files/lyft_drivers.html')
  
df['end_date'] = pd.to_datetime(df['end_date'])
df =df[df['end_date'].notnull()]
df['year'] = df['end_date'].dt.year
df['count'] = 1
df = df.groupby('year')['count'].sum().to_frame('n_churned').reset_index().sort_values('year')

df['prev_chu'] = df['n_churned'].shift(1)

df = df.fillna(0.0)

#f['rate'] = 100*(df['n_churned'] - df['prev_chu'])/df['n_churned']
#
def comparison(x):
    if x < 0:
        return 'decrease'
    elif x > 0:
        return 'increase'
    else: return 'no change'

df['diff'] = df['n_churned'] - df['prev_chu']

df['measure'] = df['diff'].apply(comparison)
res = df[['year', 'n_churned', 'prev_chu', 'measure']]
res.head()

,year,n_churned,prev_chu,measure
0,2015,5,0.0,increase
1,2016,5,5.0,no change
2,2017,8,5.0,increase
3,2018,25,8.0,increase
4,2019,7,25.0,decrease


--------------------

# Top 3 Wineries In The World
- Hard (WineMagazine)

Find the top 3 wineries in each country based on the average points earned. Output the country along with the best, second best, and third best wineries. If there is no second winery (NULL value) output 'No second winery' and if there is no third winery output 'No third winery'. For outputting wineries format them like this: "winery (avg_points)"

# Approach
- Use .groupby(column_name) to group the dataframe about the specifed column then mean() to get the number the average or mean per group; convert the result to a dataframe using to_frame('column_name')

- Create a column containing the average points rank per city by using sort_values(column_name, order) to sort along the average points, groupby() city then applying cumcount() + 1 for the rank counting

- Filter out top 3 by selecting specific column/s from dataframe using [column_name] then rows under the ranking with values less than or equal to '<=' 3

- Pivot the dataframe using pd.pivot_table with country as the index, ranking as the columns, winery as the values, and specify the aggfunc to 'first'

- Clean resulting multilevel indexes by applying reset_index()


In [90]:
from getdata_fromhtml import getdata_fromhtml as g
df = g('/home/kassa/Downloads/StrataScratch_files/winemag.html')
df['points'] = df['points'].apply(pd.to_numeric) 

import pandas as pd
import numpy as np
df = df.groupby(['country', 'winery'])['points'].mean().to_frame('winery (avg_points)').reset_index().sort_values('winery (avg_points)', ascending = False)
#resu = df.iloc[0:3, :]
df['winery_avg_points'] = df['winery'].astype(str) + ' ('+df['winery (avg_points)'].astype(str)+')'

df['ranks'] = df.groupby(['country']).cumcount() + 1
 
df = df[df['ranks'] <= 3]

res = pd.pivot_table(df, index = 'country', columns = str('ranks'), values = 'winery_avg_points', aggfunc = 'first').reset_index()

lis = ['no winery', 'no winery']
for i in range(2, 4):
    res.loc[:, i] = res.loc[:, i].fillna(lis[i-2])
    
res.head()

ranks,country,1,2,3
0,Argentina,Bodega NoemÌ_a de Patagonia (89),Bodega Norton (86),Rutini (86)
1,Australia,Madison Ridge (84),no winery,no winery
2,Austria,Schloss Gobelsburg (93),HÌ¦pler (83),no winery
3,Bulgaria,Targovishte (84),no winery,no winery
4,Chile,Santa Carolina (85),AltaÌør (85),FranÌ¤ois Lurton (85)


-----------------
# City With Most Amenities
- Hard (airbnb)

You're given a dataset of searches for properties on Airbnb. For simplicity, let's say that each search result (i.e., each row) represents a unique host. Find the city with the most amenities across all their host's properties. Output the name of the city.

# Approach

- Count the amenities by splitting after the ,

- Sum the number of amenities by city and identify the city with the most amenities

- Combine the results with the main table based on the city


In [92]:
from getdata_fromhtml import getdata_fromhtml as g
import pandas as pd

# Start writing code
df = g('/home/kassa/Downloads/StrataScratch_files/airbnb_search_details.html')
df['no_amenities'] = df['amenities'].apply(lambda x: len(x.split(','))).astype(int)
df = df.groupby('city')['no_amenities'].sum().to_frame('total').reset_index().sort_values('total', ascending = False)

res = df[df['total'] == df['total'].max()]['city']
res.head()

4    NYC
Name: city, dtype: object

-------------------
# Common Letters
- Hard (Google)

Find the top 3 most common letters across all the words from both the tables. Output the letter along with the number of occurrences and order records in descending order based on the number of occurrences.

# Approach 
- Take away each word per row and store it in a single column. Do this by using str.split(delimiter) then applying stack() to reshape the result, tolist() to convert the object to a list format. Do this for columns words1, words2 and contents
- Concatenate the list of words per column into a single list using the operator '+'
- Use 'delimeter'.join(list) to merge the values as a single string
- Split the string per letter using list()
- Convert the resulting list containing all the letters to a dataframe using pd.DataFrame(data, columns) then drop all rows with empty values
- Groupby letter then use size() to count the number of values or occurences per group

In [96]:
from getdata_fromhtml import getdata_fromhtml as g

google_file_store = g('/home/kassa/Downloads/StrataScratch_files/google_file.html')
df = g('/home/kassa/Downloads/StrataScratch_files/google_word_lists.html')

import pandas as pd

df_words1 = df['words1'].str.split(',', expand =True)
df_words2 = df['words2'].str.split(',', expand =True)
df_content = google_file_store['contents'].str.split(' ', expand = True)

df_words1  = df_words1.stack()
df_words2  = df_words2.stack()
df_content = df_content.stack()

words1 = ''.join(df_words1).lower()
words2 = ''.join(df_words2).lower()
words3 = ''.join(df_content).lower()
 
merged = words1+words2+words3

word_list = list(merged)
data = pd.DataFrame(word_list, columns = ['col_1'])
top3 = data.groupby('col_1').size().to_frame('n_occurence').reset_index().sort_values('n_occurence', ascending = False)

res = top3.iloc[0:3, :]
res.head()

,col_1,n_occurence
3,a,62
7,e,53
20,t,52


-----------------------
# Health Inspections Per Street
- Hard (City of San Farancisco)

Find the number of inspections for each street name where a risk category value is assigned. Output the result along with the street name. Order the results based on the number of inspections in descending order.

It's hard to catch every variation of how street names are written but your code should skip street names that include only 1 letter like the street named 'b' in 295 B Orizaba Ave. The letter 'b' is an element of the "house number" and not the street name. Rather than extracting 'b', Orizaba should be returned.

# Approach
- notnull() can be used to determine the non-missing values in the dataframe

- Split the address into words by converting the object to str first, using astype(str), then applying str.split()

- Use str.lower() to convert strings to lower case

- Use size() to get the number of elements in the specified column then convert the resulting object to dataframe using to_frame('column_name')

- Use sort_values(column_name, order) to sort along a specified column; Set order to False to display the printed values in descending order

In [98]:
import pandas as pd
from getdata_fromhtml import getdata_fromhtml as g
df = g('/home/kassa/Downloads/StrataScratch_files/sf_restaurant_health_violations.html')

def split_is_numeric(x):
    a = x.split(' ')
    if (str(a[0]).isdigit() == True) & (len(str(a[1])) != 1):
        return  a[1].lower()
    elif str(a[0]).isdigit() == True & len(str(a[1])) == 1:
        return a[2].lower()
    else:
        return a[0].lower()
df['street'] = df['business_address'].apply(split_is_numeric)
df = df[df['risk_category'].notna()]
resu = df.groupby('street')['inspection_date'].count().to_frame('n_inspection').reset_index().sort_values('n_inspection', ascending = False)
resu.head()

,street,n_inspection
70,mission,19
43,geary,15
87,powell,13
29,clement,13
68,market,9


-----------------------
# Find the number of inspections for each risk category by inspection type
- Hard (City of San Francisco)

Find the number of inspections that resulted in each risk category per each inspection type.
Consider the records with no risk category value belongs to a separate category.

Output the result along with the corresponding inspection type and the corresponding total number of inspections per that type. The output should be pivoted, meaning that each risk category + total number should be a separate column.

Order the result based on the number of inspections per inspection type in descending order.

In [115]:
from getdata_fromhtml import getdata_fromhtml as g
import pandas as pd

# Start writing code
df = g('/home/kassa/Downloads/StrataScratch_files/sf_restaurant_health_violations.html')

df['risk_category'] = df['risk_category'].replace(r'^\s*$', 'othes', regex = True).astype(str)
 
df = df.groupby(['inspection_type', 'risk_category']).size().to_frame('n_inspection').reset_index().sort_values('n_inspection', ascending  = False)

resu = df.pivot(index = 'inspection_type', columns = 'risk_category', values = 'n_inspection').reset_index()
resu = resu.fillna(0)
#resu
df = resu[['inspection_type', 'othes', 'Low Risk', 'Moderate Risk', 'High Risk']] 

df['total'] = df.sum(axis = 1)
result = df.sort_values('total', ascending = False)
result.head()

/usr/lib/python3/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


risk_category,inspection_type,othes,Low Risk,Moderate Risk,High Risk,total
5,Routine - Unscheduled,11.0,106.0,72.0,36.0,225.0
4,Reinspection/Followup,29.0,2.0,2.0,0.0,33.0
1,New Construction,12.0,0.0,0.0,0.0,12.0
0,Complaint,6.0,1.0,3.0,1.0,11.0
2,New Ownership,4.0,1.0,1.0,1.0,7.0


--------------
# Host Response Rates With Cleaning Fees
- Hard (Airbnb)

Find the average host response rate with a cleaning fee for each zipcode. Present the results as a percentage along with the zip code value.  Convert the column 'host_response_rate' from TEXT to NUMERIC using type casts and string processing (take missing values as NULL). Order the result in ascending order based on the average host response rater after cleaning.

# Approach

- Use replace() to search a string for a specified value, or a regular expression, and returns a new string where the specified values are replaced

 - Use a lambda (lambda x: x[column] == condition, result) function to apply an operation for each row values on a specified column

 - notnull() can be used to determine the non-missing values in the dataframe

 - Use [ [ column_name/s] ] to return a specified column of the dataframe

In [130]:
from getdata_fromhtml import getdata_fromhtml as g
import pandas as pd

# Start writing code
df = g('/home/kassa/Downloads/StrataScratch_files/airbnb_search_details.html')
df['host_response_rate'] = df['host_response_rate'].str.rstrip('%').apply(pd.to_numeric)
 
df = df[df['host_response_rate'].notnull()]
df = df[df['cleaning_fee'].apply(lambda x: x == 'TRUE')]
df = df.groupby('zipcode')['host_response_rate'].mean().reset_index().sort_values('host_response_rate', ascending = True)

df.head()

,zipcode,host_response_rate
58,91324,0.0
31,90028,25.0
5,10035,67.0
52,90703,70.0
7,10039,76.0


------------------------
# Reviews Bins on Reviews Number
- Hard (airbnb)

To better understand the effect of the review count on the price, categorize the number of reviews into the following groups along with the price.

    0 reviews: NO
    1 to 5 reviews: FEW
    6 to 15 reviews: SOME
    16 to 40 reviews: MANY
    more than 40 reviews: A LOT
   
# Approach

- Define variables which contains the conditions to be satisfied (if num_reviews == 0, etc), and the result ('NO', 'FEW', etc) once the conditions are satisfied

- Use between(limit 1, limit 2) to test if the values in the column is between the boundary values left and right

- Use np.select(condition, result) to return  resulting values that met the conditions specified

- Use [ [ column_name/s] ] to return a specified column of the dataframe


In [133]:
from getdata_fromhtml import getdata_fromhtml as g
import pandas as pd

# Start writing code
df = g('/home/kassa/Downloads/StrataScratch_files/airbnb_search_details.html')
df['number_of_reviews'] = df['number_of_reviews'].apply(pd.to_numeric)
df['price'] = df['price'].apply(pd.to_numeric)
def bins_reviews(x):
    if x == 0: return 'NO'
    elif 1<= x <= 5: return 'FEW'
    elif 6<= x <= 15: return 'SOME'
    elif 16 <= x <= 40: return 'MANY'
    elif x >40: return 'A LOT'

df['new_review'] = df['number_of_reviews'].apply(bins_reviews)
res = df[['new_review', 'price']]
#res = res.groupby('new_review')['price'].sum().reset_index()

res.head()

,new_review,price
0,FEW,555.68
1,SOME,366.36
2,A LOT,482.83
3,SOME,448.86
4,FEW,506.89
